In [ ]:
import skimage
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import time
import argparse
import cv2
from scipy import io
from tqdm.notebook import tqdm

import torch
from torch import nn
import torch.nn.functional as F
import torchvision.models as models
import torch.optim.lr_scheduler as lr_scheduler
from pytorch_msssim import ssim

from modules import utils
from modules.models import INR
from torchsummary import summary
from modules.encoding import Encoding
from encoding import MultiResHashGrid
import lpips
from pytorch_msssim import ssim

def get_noisy_image(img_np, sigma):
    """Adds Gaussian noise to an image.
    
    Args: 
        img_np: image, np.array with values from 0 to 1
        sigma: std of the noise
    """
    img_noisy_np = np.clip(img_np + np.random.normal(scale=sigma, size=img_np.shape), 0, 1).astype(np.float32)

    return img_noisy_np

lpips_vgg_model = lpips.LPIPS(net="vgg")
lpips_alex_model = lpips.LPIPS(net="alex")
folder_path = '/home/wrt/Poly/INCODE-main/data/image'

parser = argparse.ArgumentParser(description='INCODE')
# Shared Parameters
parser.add_argument('--input',type=str, default='/root/autodl-tmp/INCODE-main/data/0640.png', help='Input image path')
# parser.add_argument('--input',type=str, default='/home/wrt/Poly/deep-image/data/denoising/F16_GT.png', help='Input image path')
parser.add_argument('--inr_model',type=str, default='incode', help='[gauss, mfn, relu, siren, wire, wire2d, ffn, incode]')
parser.add_argument('--lr',type=float, default=5e-4, help='Learning rate')
parser.add_argument('--using_schedular', type=bool, default=True, help='Whether to use schedular')
parser.add_argument('--scheduler_b', type=float, default=0.01, help='LR scheduler [set it from 0.05 to 0.1]')
parser.add_argument('--maxpoints', type=int, default=512*512, help='Batch size')
parser.add_argument('--niters', type=int, default=2001, help='Number if iterations')
parser.add_argument('--steps_til_summary', type=int, default=10, help='Number of steps till summary visualization')
# INCODE Parameters
parser.add_argument('--a_coef',type=float, default=0.1993, help='a coeficient')
parser.add_argument('--b_coef',type=float, default=0.0196, help='b coeficient')
parser.add_argument('--c_coef',type=float, default=0.0588, help='c coeficient')
parser.add_argument('--d_coef',type=float, default=0.0269, help='d coeficient')
args = parser.parse_args(args=[])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

sigma=25/255


print(args.input)
## Loading Data
# No Encoding
pos_encode_no = {'type': None}
im = utils.normalize(plt.imread(args.input).astype(np.float32), True)
im = cv2.resize(im, None, fx=1/4, fy=1/4, interpolation=cv2.INTER_AREA)
H, W, _ = im.shape
# Frequency Encoding
pos_encode_freq = {'type':'frequency', 'use_nyquist': True, 'mapping_input': int(max(H, W))}

# Gaussian Encoding
pos_encode_gaus = {'type':'gaussian', 'scale_B': 10, 'mapping_input': 256}
# Create a noisy image using realistic sensor measurement

im_noisy = get_noisy_image(im, sigma)
im_noisy_gt = utils.normalize(im_noisy, True)[None, ...].astype(np.float32)
noise_psnr = ((im_noisy_gt - im)**2).mean().item()
noise_psnr = -10*np.log10(noise_psnr)

print('Noise PSNR:', noise_psnr,'\n')
print('H:',H, 'W:',W)


model_psnr={}
model_noise_psnr={}
model_ssim={}
total_time = {}




In [ ]:
cv2.imwrite('/root/autodl-tmp/INCODE-main/im.png',im[:,:, ::-1]*255)

In [ ]:
# # Create a noisy image using realistic sensor measurement
# tau = 9e1 # Photon noise
# noise_snr = 2 # Readout noise

# sigma=500/255
# # im_noisy = utils.measure(im, noise_snr, tau)
# im_noisy = get_noisy_image(im, sigma)
# im_noisy_gt = utils.normalize(im_noisy, True)[None, ...].astype(np.float32)
# noise_psnr = ((im_noisy_gt - im)**2).mean().item()
# noise_psnr = -10*np.log10(noise_psnr)

# print('Noise PSNR:', noise_psnr,'\n')
# # Plot
# im_noisy = (im_noisy - im_noisy.min()) / (im_noisy.max() - im_noisy.min())

# fig, axes = plt.subplots(1, 2, figsize=(10, 10))
# axes[0].set_title('Ground Truth')
# axes[0].imshow(im)
# axes[0].axis('off')
# axes[1].set_title('Noisy Image')
# axes[1].imshow(im_noisy)
# axes[1].axis('off')
# plt.show()

In [ ]:
from skimage.metrics import structural_similarity as compare_ssim
original_img = torch.tensor(im).permute(2, 0, 1).unsqueeze(0)
reconstruct_img = torch.tensor(im_noisy).permute(2, 0, 1).unsqueeze(0)
mse_array_noisy = ((original_img - reconstruct_img)**2).mean().item()
noise_psnr = -10*np.log10(mse_array_noisy)
ms_ssim = ssim(original_img, reconstruct_img, data_range=1, size_average=False)
original_img = im
reconstruct_img = im_noisy
ms_ssim = compare_ssim(im, im_noisy, data_range=1, size_average=False,channel_axis=-1)
print('noise_psnr:', noise_psnr)
print('noise_ssim:', ms_ssim)

In [ ]:
# InstantNGP
class SineLayer(nn.Module):
    '''
    SineLayer is a custom PyTorch module that applies the Sinusoidal activation function to the output of a linear transformation.

    Args:
        in_features (int): Number of input features.
        out_features (int): Number of output features.
        bias (bool, optional): If True, the linear transformation includes a bias term. Default is True.
        is_first (bool, optional): If it is the first layer, we initialize the weights differently. Default is False.
        omega_0 (float, optional): Frequency scaling factor for the sinusoidal activation. Default is 30.
        scale (float, optional): Scaling factor for the output of the sine activation. Default is 10.0.
        init_weights (bool, optional): If True, initializes the layer's weights according to the SIREN paper. Default is True.

    '''
    
    def __init__(self, in_features, out_features, bias=True,
                is_first=False, omega_0=30, scale=10.0, init_weights=True):
        super().__init__()
        self.omega_0 = omega_0
        self.is_first = is_first
        
        self.in_features = in_features
        self.linear = nn.Linear(in_features, out_features, bias=bias)
        
        # if init_weights:
        #     self.init_weights()
    
    def init_weights(self):
        # self.linear.bias.data.fill_(10)
        with torch.no_grad():
            if self.is_first:
                self.linear.weight.uniform_(-1 / self.in_features, 
                                            1 / self.in_features)      
            else:
                self.linear.weight.uniform_(-np.sqrt(1 / self.in_features), 
                                            np.sqrt(1 / self.in_features))
        
    def forward(self, input):
        return self.linear(input)

class Siren(nn.Module):
    """
        Siren activation
        https://arxiv.org/abs/2006.09661
    """

    def __init__(self, w0=1):
        """
            w0 comes from the end of section 3
            it should be 30 for the first layer
            and 1 for the rest
        """
        super().__init__()
        self.w0 = torch.tensor(w0)

    def forward(self, x):
        # return torch.sin(self.w0*(torch.abs(x)+1)*x) 
        return torch.sin(self.w0 * x) 
    def extra_repr(self):
        return "w0={}".format(self.w0)
    
class PolyReLUCode(nn.Module):
    
    def __init__(
        self,activate='ReLU',norm_type='None'
        ) -> None:
        super(PolyReLUCode, self).__init__()
        input_dim = 32
        hidden_channel=64
        
        self.positional_encoding = MultiResHashGrid(dim=2,
                                                                n_levels = 16,
                                                                n_features_per_level = 2,
                                                                log2_hashmap_size = 15,
                                                                base_resolution = 16,
                                                                finest_resolution = 256,
                                                            )
        w1=100
        w2=2
        w3=1
        w4=1
        w5=1
        w6=1
        self.linear1=SineLayer(input_dim,hidden_channel,omega_0=w1,is_first=True)
        if norm_type=='LayerNorm':
            self.norm1=nn.LayerNorm(hidden_channel)
            self.norm2=nn.LayerNorm(hidden_channel)
            self.norm3=nn.LayerNorm(hidden_channel)
            self.norm4=nn.LayerNorm(hidden_channel)
            self.norm5=nn.LayerNorm(hidden_channel)
            self.norm6=nn.LayerNorm(hidden_channel)
        elif norm_type=='BatchNorm1d':
            self.norm1=nn.BatchNorm1d(65536)
            self.norm2=nn.BatchNorm1d(65536)
            self.norm3=nn.BatchNorm1d(65536)
            self.norm4=nn.BatchNorm1d(65536)
        elif norm_type=='None':
            self.norm1=nn.Identity()
            self.norm2=nn.Identity()
            self.norm3=nn.Identity()
            
        self.linear2=SineLayer(hidden_channel,hidden_channel,omega_0=w2)
        # self.linear3=SineLayer(hidden_channel,hidden_channel,omega_0=w3)
        if activate=='ReLU':
            self.nolinear1=nn.ReLU()
            self.nolinear2=nn.ReLU()
            self.nolinear3=nn.ReLU()
            self.nolinear4=nn.ReLU()
            self.nolinear5=nn.ReLU()
            self.nolinear6=nn.ReLU()
        if activate=='Siren':
            self.nolinear1=Siren(w1)
            self.nolinear2=Siren(w2)
            self.nolinear3=Siren(w3)
            self.nolinear4=Siren(w4)
            self.nolinear5=Siren(1)
            self.nolinear6=Siren(1)
        layers = []
        layers.append(SineLayer(hidden_channel, 3,is_first=False))
        # layers.append(nn.Sigmoid())
        self.layers = nn.Sequential(*layers)
    def forward(self, input):

        x = input
        x=self.positional_encoding(x)
        x = self.nolinear1(self.linear1(x))
        x = self.nolinear2(self.linear2(x))
        # x = self.nolinear3(self.linear3(x))
        x = self.layers(x)
        
        return x
    

### InstantNGP
args.inr_model='relu'

args.lr=5e-3
args.scheduler_b=0.1
model = PolyReLUCode(activate='ReLU',norm_type='LayerNorm').to(device)

# print(model)
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total number of parameters: {num_params/1e6}(M)')



# Optimizer setup
if args.inr_model == 'wire':
    args.lr = args.lr * min(1, args.maxpoints / (H * W))
optim = torch.optim.Adam(lr=args.lr, params=model.parameters())
scheduler = lr_scheduler.LambdaLR(optim, lambda x: args.scheduler_b ** min(x / args.niters, 1))

# Initialize lists for PSNR and MSE values
mse_array = torch.zeros(args.niters, device=device)
mse_array_noisy = torch.zeros(args.niters, device=device)

# Initialize best loss value as positive infinity
best_loss = torch.tensor(float('inf'))

# Generate coordinate grid
coords = utils.get_coords(H, W, dim=2)[None, ...]

# Convert input image to a tensor and reshape
gt = torch.tensor(im).cuda().reshape(H * W, 3)[None, ...]
gt_noisy = torch.tensor(im_noisy).cuda().reshape(H*W, 3)[None, ...]


# Initialize a tensor for reconstructed data
rec = torch.zeros_like(gt)
psnr_values = []
noise_psnr_values=[]
ssim_values = []
cumulative_times = []
start_time = time.time()  
for step in tqdm(range(args.niters)):
    # Randomize the order of data points for each iteration
    indices = torch.randperm(H*W)

    # Process data points in batches
    for b_idx in range(0, H*W, args.maxpoints):
        b_indices = indices[b_idx:min(H*W, b_idx+args.maxpoints)]
        b_coords = coords[:, b_indices, ...].cuda()
        b_indices = b_indices.cuda()
        
        # Calculate model output
        if args.inr_model == 'incode':
            model_output, coef = model(b_coords)  
        else:
            model_output = model(b_coords) 

        # Update the reconstructed data
        with torch.no_grad():
            rec[:, b_indices, :] = model_output

        # Calculate the output loss
        output_loss = ((model_output - gt_noisy[:, b_indices, :])**2).mean()
        
        if args.inr_model == 'incode':
            # Calculate regularization loss for 'incode' model
            a_coef, b_coef, c_coef, d_coef = coef[0]  
            reg_loss = args.a_coef * torch.relu(-a_coef) + \
                    args.b_coef * torch.relu(-b_coef) + \
                    args.c_coef * torch.relu(-c_coef) + \
                    args.d_coef * torch.relu(-d_coef)

            # Total loss for 'incode' model
            loss = output_loss + reg_loss 
        else: 
            # Total loss for other models
            loss = output_loss

        # Perform backpropagation and update model parameters
        optim.zero_grad()
        loss.backward()
        optim.step()

    # Calculate and log mean squared error (MSE) and PSNR
    with torch.no_grad():
        mse_array_noisy[step] = ((gt_noisy - rec)**2).mean().item()
        mse_array[step] = ((gt - rec)**2).mean().item()
        psnr = -10*torch.log10(mse_array[step])
        noise_psnr = -10*torch.log10(mse_array_noisy[step])
        noise_psnr_values.append(noise_psnr.item())
        psnr_values.append(psnr.item())
        
        
    # Adjust learning rate using a scheduler if applicable
    if args.using_schedular:
        scheduler.step()

    # Prepare reconstructed image for visualization
    imrec = rec[0, ...].reshape(H, W, 3).detach().cpu().numpy()
    current_total_time = time.time() - start_time
    # 将当前的累积时间添加到列表中
    cumulative_times.append(current_total_time)
    # Check if the current iteration's MSE is the best so far
    if (mse_array[step] < best_loss) or (step == 0):
        best_loss = mse_array[step]
        best_img = imrec
        best_img = (best_img - best_img.min()) / (best_img.max() - best_img.min())
    original_img = torch.tensor(im).permute(2, 0, 1).unsqueeze(0)
    reconstruct_img = torch.tensor(best_img).permute(2, 0, 1).unsqueeze(0)
    ms_ssim = ssim(original_img, reconstruct_img, data_range=1, size_average=False)
    ssim_values.append(ms_ssim.item())
    # Display intermediate results at specified intervals
    if step % args.steps_til_summary == 0:
        print("Epoch: {} | Total Loss: {:.5f} | Truth PSNR: {:.4f} | Noise PSNR: {:.4f}".format(step, 
                                                                    mse_array_noisy[step].item(),
                                                                    psnr.item(),noise_psnr.item())) 
    # if step % 1000 == 0:
# Plot
im_noisy = (im_noisy - im_noisy.min()) / (im_noisy.max() - im_noisy.min())

fig, axes = plt.subplots(1, 3, figsize=(10, 10))
axes[0].set_title('Ground Truth')
axes[0].imshow(im)
axes[0].axis('off')
axes[1].set_title('Noisy Image')
axes[1].imshow(im_noisy)
axes[1].axis('off')
axes[2].set_title('Denoised')
axes[2].imshow(best_img)
axes[2].axis('off')
plt.show()

        
# Print maximum PSNR achieved during training
print('--------------------')
print('Max PSNR:', max(psnr_values))
print('Max SSIM:', max(ssim_values))
print('--------------------')
args.inr_model='InstantNGP'
model_psnr[args.inr_model]=psnr_values
total_time[args.inr_model]=np.array(cumulative_times)
model_ssim[args.inr_model]=ssim_values
model_noise_psnr[args.inr_model]=noise_psnr_values



In [ ]:
best_img_save_name='/root/autodl-tmp/INCODE-main/result/image_denoise/'+args.inr_model+'_best_img.png'
cv2.imwrite('/root/autodl-tmp/INCODE-main/result/image_denoise/im.png',im[:,:, ::-1]*255)
cv2.imwrite('/root/autodl-tmp/INCODE-main/result/image_denoise/im_noisy.png',im_noisy[:,:, ::-1]*255)
cv2.imwrite(best_img_save_name,best_img[:,:, ::-1]*255)
tmax=0.15
plt.style.use('default')

image1 = cv2.imread('/root/autodl-tmp/INCODE-main/result/image_denoise/im_noisy.png')
image2 = cv2.imread(best_img_save_name)

# 确保图像为同一尺寸
image1 = cv2.resize(image1, (image2.shape[1], image2.shape[0]))

# 计算差异
difference = cv2.absdiff(image1, image2)

# 将差异转换为灰度图，以便更清晰地看到差异
gray_difference = cv2.cvtColor(difference, cv2.COLOR_BGR2GRAY)/255

# 显示差异
plt.figure(figsize=(20, 20))
plt.subplot(1, 3, 1), plt.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
plt.title('Image 1'), plt.xticks([]), plt.yticks([])

plt.subplot(1, 3, 2), plt.imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
plt.title('Image 2'), plt.xticks([]), plt.yticks([])

plt.subplot(1, 3, 3), plt.imshow(gray_difference, cmap='jet',vmin=0,vmax=tmax)
plt.title('Difference'), plt.xticks([]), plt.yticks([])

best_img_save_name='/root/autodl-tmp/INCODE-main/result/image_denoise/'+args.inr_model+'_best_img_inferno.png'
fig = plt.figure()
plt.imshow(gray_difference, cmap='inferno',vmin=0,vmax=tmax)
plt.axis('off')
# plt.savefig(best_img_save_name,format='png',dpi=2000,bbox_inches='tight', pad_inches=0)
plt.show()


In [ ]:
#   Wire
### INCODE
### Harmonizer Configurations
MLP_configs={'task': 'denoising',
            'model': 'resnet34',
            'truncated_layer':5,
            'in_channels': 64,             
            'hidden_channels': [32, 16, 8, 4],
            'mlp_bias':0.0005,
            'activation_layer': nn.SiLU,
            'GT': torch.tensor(im_noisy_gt).to(device).permute(0, 3, 1, 2)
            }
# WIRE
### Model Configurations
args.inr_model='wire'
args.lr= 5e-3
args.scheduler_b=0.1
model = INR(args.inr_model).run(in_features=2,
                                out_features=3, 
                                hidden_features=256,
                                hidden_layers=3,
                                first_omega_0=5.0,
                                hidden_omega_0=5.0,
                                pos_encode_configs=pos_encode_no
                                ).to(device)
# print(model)
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total number of parameters: {num_params/1e6}(M)')

# Optimizer setup
if args.inr_model == 'wire':
    args.lr = args.lr * min(1, args.maxpoints / (H * W))
optim = torch.optim.Adam(lr=args.lr, params=model.parameters())
scheduler = lr_scheduler.LambdaLR(optim, lambda x: args.scheduler_b ** min(x / args.niters, 1))

# Initialize lists for PSNR and MSE values
mse_array = torch.zeros(args.niters, device=device)
mse_array_noisy = torch.zeros(args.niters, device=device)

# Initialize best loss value as positive infinity
best_loss = torch.tensor(float('inf'))

# Generate coordinate grid
coords = utils.get_coords(H, W, dim=2)[None, ...]

# Convert input image to a tensor and reshape
gt = torch.tensor(im).cuda().reshape(H * W, 3)[None, ...]
gt_noisy = torch.tensor(im_noisy).cuda().reshape(H*W, 3)[None, ...]


# Initialize a tensor for reconstructed data
rec = torch.zeros_like(gt)
psnr_values = []
noise_psnr_values=[]
ssim_values = []
cumulative_times = []
start_time = time.time()  
for step in tqdm(range(args.niters)):
    # Randomize the order of data points for each iteration
    indices = torch.randperm(H*W)

    # Process data points in batches
    for b_idx in range(0, H*W, args.maxpoints):
        b_indices = indices[b_idx:min(H*W, b_idx+args.maxpoints)]
        b_coords = coords[:, b_indices, ...].cuda()
        b_indices = b_indices.cuda()
        
        # Calculate model output
        if args.inr_model == 'incode':
            model_output, coef = model(b_coords)  
        else:
            model_output = model(b_coords) 

        # Update the reconstructed data
        with torch.no_grad():
            rec[:, b_indices, :] = model_output

        # Calculate the output loss
        output_loss = ((model_output - gt_noisy[:, b_indices, :])**2).mean()
        
        if args.inr_model == 'incode':
            # Calculate regularization loss for 'incode' model
            a_coef, b_coef, c_coef, d_coef = coef[0]  
            reg_loss = args.a_coef * torch.relu(-a_coef) + \
                    args.b_coef * torch.relu(-b_coef) + \
                    args.c_coef * torch.relu(-c_coef) + \
                    args.d_coef * torch.relu(-d_coef)

            # Total loss for 'incode' model
            loss = output_loss + reg_loss 
        else: 
            # Total loss for other models
            loss = output_loss

        # Perform backpropagation and update model parameters
        optim.zero_grad()
        loss.backward()
        optim.step()

    # Calculate and log mean squared error (MSE) and PSNR
    with torch.no_grad():
        mse_array_noisy[step] = ((gt_noisy - rec)**2).mean().item()
        mse_array[step] = ((gt - rec)**2).mean().item()
        psnr = -10*torch.log10(mse_array[step])
        noise_psnr = -10*torch.log10(mse_array_noisy[step])
        noise_psnr_values.append(noise_psnr.item())
        psnr_values.append(psnr.item())
        
        
    # Adjust learning rate using a scheduler if applicable
    if args.using_schedular:
        scheduler.step()

    # Prepare reconstructed image for visualization
    imrec = rec[0, ...].reshape(H, W, 3).detach().cpu().numpy()
    current_total_time = time.time() - start_time
    # 将当前的累积时间添加到列表中
    cumulative_times.append(current_total_time)
    # Check if the current iteration's MSE is the best so far
    if (mse_array[step] < best_loss) or (step == 0):
        best_loss = mse_array[step]
        best_img = imrec
        best_img = (best_img - best_img.min()) / (best_img.max() - best_img.min())
    original_img = torch.tensor(im).permute(2, 0, 1).unsqueeze(0)
    reconstruct_img = torch.tensor(best_img).permute(2, 0, 1).unsqueeze(0)
    ms_ssim = ssim(original_img, reconstruct_img, data_range=1, size_average=False)
    ssim_values.append(ms_ssim.item())
    # Display intermediate results at specified intervals
    if step % args.steps_til_summary == 0:
        print("Epoch: {} | Total Loss: {:.5f} | Truth PSNR: {:.4f} | Noise PSNR: {:.4f}".format(step, 
                                                                    mse_array_noisy[step].item(),
                                                                    psnr.item(),noise_psnr.item())) 
    # if step % 1000 == 0:
# Plot
im_noisy = (im_noisy - im_noisy.min()) / (im_noisy.max() - im_noisy.min())

fig, axes = plt.subplots(1, 3, figsize=(10, 10))
axes[0].set_title('Ground Truth')
axes[0].imshow(im)
axes[0].axis('off')
axes[1].set_title('Noisy Image')
axes[1].imshow(im_noisy)
axes[1].axis('off')
axes[2].set_title('Denoised')
axes[2].imshow(best_img)
axes[2].axis('off')
plt.show()

        
# Print maximum PSNR achieved during training
print('--------------------')
print('Max PSNR:', max(psnr_values))
print('Max SSIM:', max(ssim_values))
print('--------------------')
args.inr_model='wire'
model_psnr[args.inr_model]=psnr_values
total_time[args.inr_model]=np.array(cumulative_times)
model_ssim[args.inr_model]=ssim_values
model_noise_psnr[args.inr_model]=noise_psnr_values
# 将图像转换为PyTorch的Tensor格式

# =使用LPIPS模型计算距离
vgg_distance = lpips_vgg_model(original_img, reconstruct_img)
alex_distance = lpips_alex_model(original_img, reconstruct_img)
print("VGG: LPIPS distance:", vgg_distance.item())
print("ALEX: LPIPS distance:", alex_distance.item())



In [ ]:
best_img_save_name='/root/autodl-tmp/INCODE-main/result/image_denoise/'+args.inr_model+'_best_img.png'
cv2.imwrite('/root/autodl-tmp/INCODE-main/result/image_denoise/im.png',im[:,:, ::-1]*255)
cv2.imwrite('/root/autodl-tmp/INCODE-main/result/image_denoise/im_noisy.png',im_noisy[:,:, ::-1]*255)
cv2.imwrite(best_img_save_name,best_img[:,:, ::-1]*255)
tmax=0.15
plt.style.use('default')

image1 = cv2.imread('/root/autodl-tmp/INCODE-main/result/image_denoise/im_noisy.png')
image2 = cv2.imread(best_img_save_name)

# 确保图像为同一尺寸
image1 = cv2.resize(image1, (image2.shape[1], image2.shape[0]))

# 计算差异
difference = cv2.absdiff(image1, image2)

# 将差异转换为灰度图，以便更清晰地看到差异
gray_difference = cv2.cvtColor(difference, cv2.COLOR_BGR2GRAY)/255

# 显示差异
plt.figure(figsize=(20, 20))
plt.subplot(1, 3, 1), plt.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
plt.title('Image 1'), plt.xticks([]), plt.yticks([])

plt.subplot(1, 3, 2), plt.imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
plt.title('Image 2'), plt.xticks([]), plt.yticks([])

plt.subplot(1, 3, 3), plt.imshow(gray_difference, cmap='jet',vmin=0,vmax=tmax)
plt.title('Difference'), plt.xticks([]), plt.yticks([])

best_img_save_name='/root/autodl-tmp/INCODE-main/result/image_denoise/'+args.inr_model+'_best_img_inferno.png'
fig = plt.figure()
plt.imshow(gray_difference, cmap='inferno',vmin=0,vmax=tmax)
plt.axis('off')
# plt.savefig(best_img_save_name,format='png',dpi=2000,bbox_inches='tight', pad_inches=0)
plt.show()


In [ ]:
# INCODE

### INCODE
### Harmonizer Configurations
MLP_configs={'task': 'denoising',
            'model': 'resnet34',
            'truncated_layer':5,
            'in_channels': 64,             
            'hidden_channels': [32, 16, 8, 4],
            'mlp_bias':0.0005,
            'activation_layer': nn.SiLU,
            'GT': torch.tensor(im_noisy_gt).to(device).permute(0, 3, 1, 2)
            }
args.lr= 8e-4
args.scheduler_b=0.01
### Model Configurations
args.inr_model='incode'
model = INR(args.inr_model).run(in_features=2,
                                out_features=3, 
                                hidden_features=256,
                                hidden_layers=3,
                                first_omega_0=10.0, # Change it from 5 to 20 according to the noise level; higher noise -> lower omega values)
                                hidden_omega_0=30.0,
                                pos_encode_configs=pos_encode_no, 
                                MLP_configs = MLP_configs
                            ).to(device)

# print(model)
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total number of parameters: {num_params/1e6}(M)')


# Optimizer setup
if args.inr_model == 'wire':
    args.lr = args.lr * min(1, args.maxpoints / (H * W))
optim = torch.optim.Adam(lr=args.lr, params=model.parameters())
scheduler = lr_scheduler.LambdaLR(optim, lambda x: args.scheduler_b ** min(x / args.niters, 1))

# Initialize lists for PSNR and MSE values
mse_array = torch.zeros(args.niters, device=device)
mse_array_noisy = torch.zeros(args.niters, device=device)

# Initialize best loss value as positive infinity
best_loss = torch.tensor(float('inf'))

# Generate coordinate grid
coords = utils.get_coords(H, W, dim=2)[None, ...]

# Convert input image to a tensor and reshape
gt = torch.tensor(im).cuda().reshape(H * W, 3)[None, ...]
gt_noisy = torch.tensor(im_noisy).cuda().reshape(H*W, 3)[None, ...]


# Initialize a tensor for reconstructed data
rec = torch.zeros_like(gt)
psnr_values = []
noise_psnr_values=[]
ssim_values = []
cumulative_times = []
start_time = time.time()  
for step in tqdm(range(args.niters)):
    # Randomize the order of data points for each iteration
    indices = torch.randperm(H*W)

    # Process data points in batches
    for b_idx in range(0, H*W, args.maxpoints):
        b_indices = indices[b_idx:min(H*W, b_idx+args.maxpoints)]
        b_coords = coords[:, b_indices, ...].cuda()
        b_indices = b_indices.cuda()
        
        # Calculate model output
        if args.inr_model == 'incode':
            model_output, coef = model(b_coords)  
        else:
            model_output = model(b_coords) 

        # Update the reconstructed data
        with torch.no_grad():
            rec[:, b_indices, :] = model_output

        # Calculate the output loss
        output_loss = ((model_output - gt_noisy[:, b_indices, :])**2).mean()
        
        if args.inr_model == 'incode':
            # Calculate regularization loss for 'incode' model
            a_coef, b_coef, c_coef, d_coef = coef[0]  
            reg_loss = args.a_coef * torch.relu(-a_coef) + \
                    args.b_coef * torch.relu(-b_coef) + \
                    args.c_coef * torch.relu(-c_coef) + \
                    args.d_coef * torch.relu(-d_coef)

            # Total loss for 'incode' model
            loss = output_loss + reg_loss 
        else: 
            # Total loss for other models
            loss = output_loss

        # Perform backpropagation and update model parameters
        optim.zero_grad()
        loss.backward()
        optim.step()

    # Calculate and log mean squared error (MSE) and PSNR
    with torch.no_grad():
        mse_array_noisy[step] = ((gt_noisy - rec)**2).mean().item()
        mse_array[step] = ((gt - rec)**2).mean().item()
        psnr = -10*torch.log10(mse_array[step])
        noise_psnr = -10*torch.log10(mse_array_noisy[step])
        noise_psnr_values.append(noise_psnr.item())
        psnr_values.append(psnr.item())
        
        
    # Adjust learning rate using a scheduler if applicable
    if args.using_schedular:
        scheduler.step()

    # Prepare reconstructed image for visualization
    imrec = rec[0, ...].reshape(H, W, 3).detach().cpu().numpy()
    current_total_time = time.time() - start_time
    # 将当前的累积时间添加到列表中
    cumulative_times.append(current_total_time)
    # Check if the current iteration's MSE is the best so far
    if (mse_array[step] < best_loss) or (step == 0):
        best_loss = mse_array[step]
        best_img = imrec
        best_img = (best_img - best_img.min()) / (best_img.max() - best_img.min())
    original_img = torch.tensor(im).permute(2, 0, 1).unsqueeze(0)
    reconstruct_img = torch.tensor(best_img).permute(2, 0, 1).unsqueeze(0)
    ms_ssim = ssim(original_img, reconstruct_img, data_range=1, size_average=False)
    ssim_values.append(ms_ssim.item())
    # Display intermediate results at specified intervals
    if step % args.steps_til_summary == 0:
        print("Epoch: {} | Total Loss: {:.5f} | Truth PSNR: {:.4f} | Noise PSNR: {:.4f}".format(step, 
                                                                    mse_array_noisy[step].item(),
                                                                    psnr.item(),noise_psnr.item())) 
# if step % 1000 == 0:
# # Plot
im_noisy = (im_noisy - im_noisy.min()) / (im_noisy.max() - im_noisy.min())

fig, axes = plt.subplots(1, 3, figsize=(10, 10))
axes[0].set_title('Ground Truth')
axes[0].imshow(im)
axes[0].axis('off')
axes[1].set_title('Noisy Image')
axes[1].imshow(im_noisy)
axes[1].axis('off')
axes[2].set_title('Denoised')
axes[2].imshow(best_img)
axes[2].axis('off')
plt.show()

        
# Print maximum PSNR achieved during training
print('--------------------')
print('Max PSNR:', max(psnr_values))
print('Max SSIM:', max(ssim_values))
print('--------------------')
args.inr_model='incode'
model_psnr[args.inr_model]=psnr_values
total_time[args.inr_model]=np.array(cumulative_times)
model_ssim[args.inr_model]=ssim_values
model_noise_psnr[args.inr_model]=noise_psnr_values
# 将图像转换为PyTorch的Tensor格式


In [ ]:
best_img_save_name='/root/autodl-tmp/INCODE-main/result/image_denoise/'+args.inr_model+'_best_img.png'
cv2.imwrite('/root/autodl-tmp/INCODE-main/result/image_denoise/im.png',im[:,:, ::-1]*255)
cv2.imwrite('/root/autodl-tmp/INCODE-main/result/image_denoise/im_noisy.png',im_noisy[:,:, ::-1]*255)
cv2.imwrite(best_img_save_name,best_img[:,:, ::-1]*255)
tmax=0.15
plt.style.use('default')

image1 = cv2.imread('/root/autodl-tmp/INCODE-main/result/image_denoise/im_noisy.png')
image2 = cv2.imread(best_img_save_name)

# 确保图像为同一尺寸
image1 = cv2.resize(image1, (image2.shape[1], image2.shape[0]))

# 计算差异
difference = cv2.absdiff(image1, image2)

# 将差异转换为灰度图，以便更清晰地看到差异
gray_difference = cv2.cvtColor(difference, cv2.COLOR_BGR2GRAY)/255

# 显示差异
plt.figure(figsize=(20, 20))
plt.subplot(1, 3, 1), plt.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
plt.title('Image 1'), plt.xticks([]), plt.yticks([])

plt.subplot(1, 3, 2), plt.imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
plt.title('Image 2'), plt.xticks([]), plt.yticks([])

plt.subplot(1, 3, 3), plt.imshow(gray_difference, cmap='jet',vmin=0,vmax=tmax)
plt.title('Difference'), plt.xticks([]), plt.yticks([])

best_img_save_name='/root/autodl-tmp/INCODE-main/result/image_denoise/'+args.inr_model+'_best_img_inferno.png'
fig = plt.figure()
plt.imshow(gray_difference, cmap='inferno',vmin=0,vmax=tmax)
plt.axis('off')
# plt.savefig(best_img_save_name,format='png',dpi=2000,bbox_inches='tight', pad_inches=0)
plt.show()


In [ ]:
# siren
MLP_configs={'task': 'denoising',
            'model': 'resnet34',
            'truncated_layer':5,
            'in_channels': 64,             
            'hidden_channels': [32, 16, 8, 4],
            'mlp_bias':0.0005,
            'activation_layer': nn.SiLU,
            'GT': torch.tensor(im_noisy_gt).to(device).permute(0, 3, 1, 2)
            }

## Model Configurations
args.inr_model='siren'
args.lr= 1e-3
args.scheduler_b=0.1
model = INR(args.inr_model).run(in_features=2,
                                out_features=3, 
                                hidden_features=256,
                                hidden_layers=3,
                                first_omega_0=30.0,
                                hidden_omega_0=30.0,
                                pos_encode_configs=pos_encode_no
                                ).to(device)
# print(model)
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total number of parameters: {num_params/1e6}(M)')

# Optimizer setup
if args.inr_model == 'wire':
    args.lr = args.lr * min(1, args.maxpoints / (H * W))
optim = torch.optim.Adam(lr=args.lr, params=model.parameters())
scheduler = lr_scheduler.LambdaLR(optim, lambda x: args.scheduler_b ** min(x / args.niters, 1))

# Initialize lists for PSNR and MSE values
mse_array = torch.zeros(args.niters, device=device)
mse_array_noisy = torch.zeros(args.niters, device=device)

# Initialize best loss value as positive infinity
best_loss = torch.tensor(float('inf'))

# Generate coordinate grid
coords = utils.get_coords(H, W, dim=2)[None, ...]

# Convert input image to a tensor and reshape
gt = torch.tensor(im).cuda().reshape(H * W, 3)[None, ...]
gt_noisy = torch.tensor(im_noisy).cuda().reshape(H*W, 3)[None, ...]


# Initialize a tensor for reconstructed data
rec = torch.zeros_like(gt)
psnr_values = []
noise_psnr_values=[]
ssim_values = []
cumulative_times = []
start_time = time.time()  
for step in tqdm(range(args.niters)):
    # Randomize the order of data points for each iteration
    indices = torch.randperm(H*W)

    # Process data points in batches
    for b_idx in range(0, H*W, args.maxpoints):
        b_indices = indices[b_idx:min(H*W, b_idx+args.maxpoints)]
        b_coords = coords[:, b_indices, ...].cuda()
        b_indices = b_indices.cuda()
        
        # Calculate model output
        if args.inr_model == 'incode':
            model_output, coef = model(b_coords)  
        else:
            model_output = model(b_coords) 

        # Update the reconstructed data
        with torch.no_grad():
            rec[:, b_indices, :] = model_output

        # Calculate the output loss
        output_loss = ((model_output - gt_noisy[:, b_indices, :])**2).mean()
        
        if args.inr_model == 'incode':
            # Calculate regularization loss for 'incode' model
            a_coef, b_coef, c_coef, d_coef = coef[0]  
            reg_loss = args.a_coef * torch.relu(-a_coef) + \
                    args.b_coef * torch.relu(-b_coef) + \
                    args.c_coef * torch.relu(-c_coef) + \
                    args.d_coef * torch.relu(-d_coef)

            # Total loss for 'incode' model
            loss = output_loss + reg_loss 
        else: 
            # Total loss for other models
            loss = output_loss

        # Perform backpropagation and update model parameters
        optim.zero_grad()
        loss.backward()
        optim.step()

    # Calculate and log mean squared error (MSE) and PSNR
    with torch.no_grad():
        mse_array_noisy[step] = ((gt_noisy - rec)**2).mean().item()
        mse_array[step] = ((gt - rec)**2).mean().item()
        psnr = -10*torch.log10(mse_array[step])
        noise_psnr = -10*torch.log10(mse_array_noisy[step])
        noise_psnr_values.append(noise_psnr.item())
        psnr_values.append(psnr.item())
        
        
    # Adjust learning rate using a scheduler if applicable
    if args.using_schedular:
        scheduler.step()

    # Prepare reconstructed image for visualization
    imrec = rec[0, ...].reshape(H, W, 3).detach().cpu().numpy()
    current_total_time = time.time() - start_time
    # 将当前的累积时间添加到列表中
    cumulative_times.append(current_total_time)
    # Check if the current iteration's MSE is the best so far
    if (mse_array[step] < best_loss) or (step == 0):
        best_loss = mse_array[step]
        best_img = imrec
        best_img = (best_img - best_img.min()) / (best_img.max() - best_img.min())
    original_img = torch.tensor(im).permute(2, 0, 1).unsqueeze(0)
    reconstruct_img = torch.tensor(best_img).permute(2, 0, 1).unsqueeze(0)
    ms_ssim = ssim(original_img, reconstruct_img, data_range=1, size_average=False)
    ssim_values.append(ms_ssim.item())
    # Display intermediate results at specified intervals
    if step % args.steps_til_summary == 0:
        print("Epoch: {} | Total Loss: {:.5f} | Truth PSNR: {:.4f} | Noise PSNR: {:.4f}".format(step, 
                                                                    mse_array_noisy[step].item(),
                                                                    psnr.item(),noise_psnr.item())) 
    # if step % 1000 == 0:
# Plot
im_noisy = (im_noisy - im_noisy.min()) / (im_noisy.max() - im_noisy.min())

fig, axes = plt.subplots(1, 3, figsize=(10, 10))
axes[0].set_title('Ground Truth')
axes[0].imshow(im)
axes[0].axis('off')
axes[1].set_title('Noisy Image')
axes[1].imshow(im_noisy)
axes[1].axis('off')
axes[2].set_title('Denoised')
axes[2].imshow(best_img)
axes[2].axis('off')
plt.show()

        
# Print maximum PSNR achieved during training
print('--------------------')
print('Max PSNR:', max(psnr_values))
print('Max SSIM:', max(ssim_values))
print('--------------------')
args.inr_model='siren'
model_psnr[args.inr_model]=psnr_values
total_time[args.inr_model]=np.array(cumulative_times)
model_ssim[args.inr_model]=ssim_values
model_noise_psnr[args.inr_model]=noise_psnr_values
# 将图像转换为PyTorch的Tensor格式



In [ ]:
best_img_save_name='/root/autodl-tmp/INCODE-main/result/image_denoise/'+args.inr_model+'_best_img.png'
cv2.imwrite('/root/autodl-tmp/INCODE-main/result/image_denoise/im.png',im[:,:, ::-1]*255)
cv2.imwrite('/root/autodl-tmp/INCODE-main/result/image_denoise/im_noisy.png',im_noisy[:,:, ::-1]*255)
cv2.imwrite(best_img_save_name,best_img[:,:, ::-1]*255)
tmax=0.15
plt.style.use('default')

image1 = cv2.imread('/root/autodl-tmp/INCODE-main/result/image_denoise/im_noisy.png')
image2 = cv2.imread(best_img_save_name)

# 确保图像为同一尺寸
image1 = cv2.resize(image1, (image2.shape[1], image2.shape[0]))

# 计算差异
difference = cv2.absdiff(image1, image2)

# 将差异转换为灰度图，以便更清晰地看到差异
gray_difference = cv2.cvtColor(difference, cv2.COLOR_BGR2GRAY)/255

# 显示差异
plt.figure(figsize=(20, 20))
plt.subplot(1, 3, 1), plt.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
plt.title('Image 1'), plt.xticks([]), plt.yticks([])

plt.subplot(1, 3, 2), plt.imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
plt.title('Image 2'), plt.xticks([]), plt.yticks([])

plt.subplot(1, 3, 3), plt.imshow(gray_difference, cmap='jet',vmin=0,vmax=tmax)
plt.title('Difference'), plt.xticks([]), plt.yticks([])

best_img_save_name='/root/autodl-tmp/INCODE-main/result/image_denoise/'+args.inr_model+'_best_img_inferno.png'
fig = plt.figure()
plt.imshow(gray_difference, cmap='inferno',vmin=0,vmax=tmax)
plt.axis('off')
# plt.savefig(best_img_save_name,format='png',dpi=2000,bbox_inches='tight', pad_inches=0)
plt.show()


In [ ]:
# HO_siren

class SineLayer(nn.Module):
    '''
    SineLayer is a custom PyTorch module that applies the Sinusoidal activation function to the output of a linear transformation.

    Args:
        in_features (int): Number of input features.
        out_features (int): Number of output features.
        bias (bool, optional): If True, the linear transformation includes a bias term. Default is True.
        is_first (bool, optional): If it is the first layer, we initialize the weights differently. Default is False.
        omega_0 (float, optional): Frequency scaling factor for the sinusoidal activation. Default is 30.
        scale (float, optional): Scaling factor for the output of the sine activation. Default is 10.0.
        init_weights (bool, optional): If True, initializes the layer's weights according to the SIREN paper. Default is True.

    '''
    
    def __init__(self, in_features, out_features, bias=True,
                 is_first=False, omega_0=30, scale=10.0, init_weights=True):
        super().__init__()
        self.omega_0 = omega_0
        self.is_first = is_first
        
        self.in_features = in_features
        self.linear = nn.Linear(in_features, out_features, bias=bias)
        
        if init_weights:
            self.init_weights()
    
    def init_weights(self):
        # self.linear.bias.data.fill_(10)
        with torch.no_grad():
            if self.is_first:
                self.linear.weight.uniform_(-1 / self.in_features, 
                                             1 / self.in_features)  
                # self.linear.bias.data.uniform_(-0.8,0.8)    
            else:
                self.linear.weight.uniform_(-np.sqrt(6 / self.in_features) / self.omega_0, 
                                             np.sqrt(6 / self.in_features) / self.omega_0)
                # self.linear.bias.data.uniform_(-8,8)
        
    def forward(self, input):
        return self.linear(input)

class Siren(nn.Module):
    """
        Siren activation
        https://arxiv.org/abs/2006.09661
    """

    def __init__(self, w0=1):
        """
            w0 comes from the end of section 3
            it should be 30 for the first layer
            and 1 for the rest
        """
        super().__init__()
        self.w0 = torch.tensor(w0)

    def forward(self, x):
        # return torch.sin(self.w0*(torch.abs(x)+1)*x) 
        return torch.sin(self.w0 * x) 
    def extra_repr(self):
        return "w0={}".format(self.w0)
        
class PolySiren(nn.Module):
    
    def __init__(
        self,activate='ReLU',norm_type='None'
        ) -> None:
        super(PolySiren, self).__init__()
        input_dim = 2

        input_dim = 2 
        hidden_channel=256
        
        
        w1=35
        w2=1
        w3=1
        w4=1
        
        
        
        self.linear1=SineLayer(input_dim,hidden_channel,omega_0=w1,is_first=True)
        if norm_type=='LayerNorm':
            self.norm1=nn.LayerNorm(hidden_channel)
            self.norm2=nn.LayerNorm(hidden_channel)
            self.norm3=nn.LayerNorm(hidden_channel)
            self.norm4=nn.LayerNorm(hidden_channel)
            self.norm5=nn.LayerNorm(hidden_channel)
            self.norm6=nn.LayerNorm(hidden_channel)
        elif norm_type=='BatchNorm1d':
            self.norm1=nn.BatchNorm1d(65536)
            self.norm2=nn.BatchNorm1d(65536)
            self.norm3=nn.BatchNorm1d(65536)
            self.norm4=nn.BatchNorm1d(65536)
        elif norm_type=='None':
            self.norm1=nn.Identity()
            self.norm2=nn.Identity()
            self.norm3=nn.Identity()
            
            
            
        self.linear2=SineLayer(hidden_channel,hidden_channel,omega_0=w2)
        self.linear3=SineLayer(hidden_channel,hidden_channel,omega_0=w3)
        self.linear4=SineLayer(hidden_channel,hidden_channel,omega_0=w4)
        # self.linear5=SineLayer(hidden_channel,hidden_channel,omega_0=w3)
        # self.linear6=SineLayer(hidden_channel,hidden_channel,omega_0=w4)
        if activate=='ReLU':
            self.nolinear1=nn.ReLU()
            self.nolinear2=nn.ReLU()
            self.nolinear3=nn.ReLU()
        if activate=='Siren':
            self.nolinear1=Siren(w1)
            self.nolinear2=Siren(w2)
            self.nolinear3=Siren(w3)
            self.nolinear4=Siren(w4)
            self.nolinear5=Siren(1)
            self.nolinear6=Siren(1)
        layers = []
        layers.append(SineLayer(hidden_channel, 3,is_first=True))
        layers.append(nn.Sigmoid())
        self.layers = nn.Sequential(*layers)
    def forward(self, input):

        x = input

        
        x = self.nolinear1(self.linear1(x))
        x = self.nolinear2(self.norm2(x+x*self.linear2(x)))
        x = self.nolinear3(self.norm3(x+x*self.linear3(x)))
        x = self.nolinear4(self.norm4(x+x*self.linear4(x)))
        # x = self.nolinear5(self.norm5(x+x*self.linear5(x)))
        # x = self.nolinear6(self.norm6(x+x*self.linear6(x)))
        # x = self.nolinear3(x)
        x = self.layers(x)
        
        return x
            

args.inr_model = 'siren'
args.lr= 2e-3
args.scheduler_b=0.1
model = PolySiren(activate='Siren',norm_type='LayerNorm').to(device)
# print(model)
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total number of parameters: {num_params/1e6}(M)')

# Optimizer setup
if args.inr_model == 'wire':
    args.lr = args.lr * min(1, args.maxpoints / (H * W))
optim = torch.optim.Adam(lr=args.lr, params=model.parameters())
scheduler = lr_scheduler.LambdaLR(optim, lambda x: args.scheduler_b ** min(x / args.niters, 1))

# Initialize lists for PSNR and MSE values
mse_array = torch.zeros(args.niters, device=device)
mse_array_noisy = torch.zeros(args.niters, device=device)

# Initialize best loss value as positive infinity
best_loss = torch.tensor(float('inf'))

# Generate coordinate grid
coords = utils.get_coords(H, W, dim=2)[None, ...]

# Convert input image to a tensor and reshape
gt = torch.tensor(im).cuda().reshape(H * W, 3)[None, ...]
gt_noisy = torch.tensor(im_noisy).cuda().reshape(H*W, 3)[None, ...]


# Initialize a tensor for reconstructed data
rec = torch.zeros_like(gt)
psnr_values = []
noise_psnr_values=[]
ssim_values = []
cumulative_times = []
start_time = time.time()  
for step in tqdm(range(args.niters)):
    # Randomize the order of data points for each iteration
    indices = torch.randperm(H*W)

    # Process data points in batches
    for b_idx in range(0, H*W, args.maxpoints):
        b_indices = indices[b_idx:min(H*W, b_idx+args.maxpoints)]
        b_coords = coords[:, b_indices, ...].cuda()
        b_indices = b_indices.cuda()
        
        # Calculate model output
        if args.inr_model == 'incode':
            model_output, coef = model(b_coords)  
        else:
            model_output = model(b_coords) 

        # Update the reconstructed data
        with torch.no_grad():
            rec[:, b_indices, :] = model_output

        # Calculate the output loss
        output_loss = ((model_output - gt_noisy[:, b_indices, :])**2).mean()
        
        if args.inr_model == 'incode':
            # Calculate regularization loss for 'incode' model
            a_coef, b_coef, c_coef, d_coef = coef[0]  
            reg_loss = args.a_coef * torch.relu(-a_coef) + \
                    args.b_coef * torch.relu(-b_coef) + \
                    args.c_coef * torch.relu(-c_coef) + \
                    args.d_coef * torch.relu(-d_coef)

            # Total loss for 'incode' model
            loss = output_loss + reg_loss 
        else: 
            # Total loss for other models
            loss = output_loss

        # Perform backpropagation and update model parameters
        optim.zero_grad()
        loss.backward()
        optim.step()

    # Calculate and log mean squared error (MSE) and PSNR
    with torch.no_grad():
        mse_array_noisy[step] = ((gt_noisy - rec)**2).mean().item()
        mse_array[step] = ((gt - rec)**2).mean().item()
        psnr = -10*torch.log10(mse_array[step])
        noise_psnr = -10*torch.log10(mse_array_noisy[step])
        noise_psnr_values.append(noise_psnr.item())
        psnr_values.append(psnr.item())
        
        
    # Adjust learning rate using a scheduler if applicable
    if args.using_schedular:
        scheduler.step()

    # Prepare reconstructed image for visualization
    imrec = rec[0, ...].reshape(H, W, 3).detach().cpu().numpy()
    current_total_time = time.time() - start_time
    # 将当前的累积时间添加到列表中
    cumulative_times.append(current_total_time)
    # Check if the current iteration's MSE is the best so far
    if (mse_array[step] < best_loss) or (step == 0):
        best_loss = mse_array[step]
        best_img = imrec
        best_img = (best_img - best_img.min()) / (best_img.max() - best_img.min())
    original_img = torch.tensor(im).permute(2, 0, 1).unsqueeze(0)
    reconstruct_img = torch.tensor(best_img).permute(2, 0, 1).unsqueeze(0)
    ms_ssim = ssim(original_img, reconstruct_img, data_range=1, size_average=False)
    ssim_values.append(ms_ssim.item())
    # Display intermediate results at specified intervals
    if step % args.steps_til_summary == 0:
        print("Epoch: {} | Total Loss: {:.5f} | Truth PSNR: {:.4f} | Noise PSNR: {:.4f}".format(step, 
                                                                    mse_array_noisy[step].item(),
                                                                    psnr.item(),noise_psnr.item())) 
    # if step % 1000 == 0:
    #     # Plot
    #     im_noisy = (im_noisy - im_noisy.min()) / (im_noisy.max() - im_noisy.min())
        
    #     fig, axes = plt.subplots(1, 3, figsize=(10, 10))
    #     axes[0].set_title('Ground Truth')
    #     axes[0].imshow(im)
    #     axes[0].axis('off')
    #     axes[1].set_title('Noisy Image')
    #     axes[1].imshow(im_noisy)
    #     axes[1].axis('off')
    #     axes[2].set_title('Denoised')
    #     axes[2].imshow(best_img)
    #     axes[2].axis('off')
    #     plt.show()

        
# Print maximum PSNR achieved during training
print('--------------------')
print('Max PSNR:', max(psnr_values))
print('Max SSIM:', max(ssim_values))
print('--------------------')
args.inr_model='HO-Siren'
model_psnr[args.inr_model]=psnr_values
total_time[args.inr_model]=np.array(cumulative_times)
model_ssim[args.inr_model]=ssim_values
model_noise_psnr[args.inr_model]=noise_psnr_values
# 将图像转换为PyTorch的Tensor格式

# =使用LPIPS模型计算距离
vgg_distance = lpips_vgg_model(original_img, reconstruct_img)
alex_distance = lpips_alex_model(original_img, reconstruct_img)
print("VGG: LPIPS distance:", vgg_distance.item())
print("ALEX: LPIPS distance:", alex_distance.item())


In [ ]:
best_img_save_name='/root/autodl-tmp/INCODE-main/result/image_denoise/'+args.inr_model+'_best_img.png'
cv2.imwrite('/root/autodl-tmp/INCODE-main/result/image_denoise/im.png',im[:,:, ::-1]*255)
cv2.imwrite('/root/autodl-tmp/INCODE-main/result/image_denoise/im_noisy.png',im_noisy[:,:, ::-1]*255)
cv2.imwrite(best_img_save_name,best_img[:,:, ::-1]*255)
tmax=0.15
plt.style.use('default')

image1 = cv2.imread('/root/autodl-tmp/INCODE-main/result/image_denoise/im_noisy.png')
image2 = cv2.imread(best_img_save_name)

# 确保图像为同一尺寸
image1 = cv2.resize(image1, (image2.shape[1], image2.shape[0]))

# 计算差异
difference = cv2.absdiff(image1, image2)

# 将差异转换为灰度图，以便更清晰地看到差异
gray_difference = cv2.cvtColor(difference, cv2.COLOR_BGR2GRAY)/255

# 显示差异
plt.figure(figsize=(20, 20))
plt.subplot(1, 3, 1), plt.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
plt.title('Image 1'), plt.xticks([]), plt.yticks([])

plt.subplot(1, 3, 2), plt.imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
plt.title('Image 2'), plt.xticks([]), plt.yticks([])

plt.subplot(1, 3, 3), plt.imshow(gray_difference, cmap='jet',vmin=0,vmax=tmax)
plt.title('Difference'), plt.xticks([]), plt.yticks([])

best_img_save_name='/root/autodl-tmp/INCODE-main/result/image_denoise/'+args.inr_model+'_best_img_inferno.png'
fig = plt.figure()
plt.imshow(gray_difference, cmap='inferno',vmin=0,vmax=tmax)
plt.axis('off')
# plt.savefig(best_img_save_name,format='png',dpi=2000,bbox_inches='tight', pad_inches=0)
plt.show()


In [ ]:
# Pos.Enc
### Model Configurations

args.inr_model='relu'
args.lr=1e-3
args.scheduler_b=0.1
model = INR(args.inr_model).run(in_features=2, 
                                hidden_features=256, 
                                hidden_layers=3, 
                                out_features=3,
                                outermost_linear=True, 
                                pos_encode_configs=pos_encode_freq
                            ).to(device)
# print(model)
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total number of parameters: {num_params/1e6}(M)')


# Optimizer setup
if args.inr_model == 'wire':
    args.lr = args.lr * min(1, args.maxpoints / (H * W))
optim = torch.optim.Adam(lr=args.lr, params=model.parameters())
scheduler = lr_scheduler.LambdaLR(optim, lambda x: args.scheduler_b ** min(x / args.niters, 1))

# Initialize lists for PSNR and MSE values
mse_array = torch.zeros(args.niters, device=device)
mse_array_noisy = torch.zeros(args.niters, device=device)

# Initialize best loss value as positive infinity
best_loss = torch.tensor(float('inf'))

# Generate coordinate grid
coords = utils.get_coords(H, W, dim=2)[None, ...]

# Convert input image to a tensor and reshape
gt = torch.tensor(im).cuda().reshape(H * W, 3)[None, ...]
gt_noisy = torch.tensor(im_noisy).cuda().reshape(H*W, 3)[None, ...]


# Initialize a tensor for reconstructed data
rec = torch.zeros_like(gt)
psnr_values = []
noise_psnr_values=[]
ssim_values = []
cumulative_times = []
start_time = time.time()  
for step in tqdm(range(args.niters)):
    # Randomize the order of data points for each iteration
    indices = torch.randperm(H*W)

    # Process data points in batches
    for b_idx in range(0, H*W, args.maxpoints):
        b_indices = indices[b_idx:min(H*W, b_idx+args.maxpoints)]
        b_coords = coords[:, b_indices, ...].cuda()
        b_indices = b_indices.cuda()
        
        # Calculate model output
        if args.inr_model == 'incode':
            model_output, coef = model(b_coords)  
        else:
            model_output = model(b_coords) 

        # Update the reconstructed data
        with torch.no_grad():
            rec[:, b_indices, :] = model_output

        # Calculate the output loss
        output_loss = ((model_output - gt_noisy[:, b_indices, :])**2).mean()
        
        if args.inr_model == 'incode':
            # Calculate regularization loss for 'incode' model
            a_coef, b_coef, c_coef, d_coef = coef[0]  
            reg_loss = args.a_coef * torch.relu(-a_coef) + \
                    args.b_coef * torch.relu(-b_coef) + \
                    args.c_coef * torch.relu(-c_coef) + \
                    args.d_coef * torch.relu(-d_coef)

            # Total loss for 'incode' model
            loss = output_loss + reg_loss 
        else: 
            # Total loss for other models
            loss = output_loss

        # Perform backpropagation and update model parameters
        optim.zero_grad()
        loss.backward()
        optim.step()

    # Calculate and log mean squared error (MSE) and PSNR
    with torch.no_grad():
        mse_array_noisy[step] = ((gt_noisy - rec)**2).mean().item()
        mse_array[step] = ((gt - rec)**2).mean().item()
        psnr = -10*torch.log10(mse_array[step])
        noise_psnr = -10*torch.log10(mse_array_noisy[step])
        noise_psnr_values.append(noise_psnr.item())
        psnr_values.append(psnr.item())
        
        
    # Adjust learning rate using a scheduler if applicable
    if args.using_schedular:
        scheduler.step()

    # Prepare reconstructed image for visualization
    imrec = rec[0, ...].reshape(H, W, 3).detach().cpu().numpy()
    current_total_time = time.time() - start_time
    # 将当前的累积时间添加到列表中
    cumulative_times.append(current_total_time)
    # Check if the current iteration's MSE is the best so far
    if (mse_array[step] < best_loss) or (step == 0):
        best_loss = mse_array[step]
        best_img = imrec
        best_img = (best_img - best_img.min()) / (best_img.max() - best_img.min())
    original_img = torch.tensor(im).permute(2, 0, 1).unsqueeze(0)
    reconstruct_img = torch.tensor(best_img).permute(2, 0, 1).unsqueeze(0)
    ms_ssim = ssim(original_img, reconstruct_img, data_range=1, size_average=False)
    ssim_values.append(ms_ssim.item())
    # Display intermediate results at specified intervals
    if step % args.steps_til_summary == 0:
        print("Epoch: {} | Total Loss: {:.5f} | Truth PSNR: {:.4f} | Noise PSNR: {:.4f}".format(step, 
                                                                    mse_array_noisy[step].item(),
                                                                    psnr.item(),noise_psnr.item())) 
    # if step % 1000 == 0:
    #     # Plot
    #     im_noisy = (im_noisy - im_noisy.min()) / (im_noisy.max() - im_noisy.min())
        
    #     fig, axes = plt.subplots(1, 3, figsize=(10, 10))
    #     axes[0].set_title('Ground Truth')
    #     axes[0].imshow(im)
    #     axes[0].axis('off')
    #     axes[1].set_title('Noisy Image')
    #     axes[1].imshow(im_noisy)
    #     axes[1].axis('off')
    #     axes[2].set_title('Denoised')
    #     axes[2].imshow(best_img)
    #     axes[2].axis('off')
    #     plt.show()

        
# Print maximum PSNR achieved during training
print('--------------------')
print('Max PSNR:', max(psnr_values))
print('Max SSIM:', max(ssim_values))
print('--------------------')
args.inr_model='Pos.Enc'
model_psnr[args.inr_model]=psnr_values
total_time[args.inr_model]=np.array(cumulative_times)
model_ssim[args.inr_model]=ssim_values
model_noise_psnr[args.inr_model]=noise_psnr_values
# 将图像转换为PyTorch的Tensor格式

# =使用LPIPS模型计算距离
vgg_distance = lpips_vgg_model(original_img, reconstruct_img)
alex_distance = lpips_alex_model(original_img, reconstruct_img)
print("VGG: LPIPS distance:", vgg_distance.item())
print("ALEX: LPIPS distance:", alex_distance.item())


In [ ]:
best_img_save_name='/root/autodl-tmp/INCODE-main/result/image_denoise/'+args.inr_model+'_best_img.png'
cv2.imwrite('/root/autodl-tmp/INCODE-main/result/image_denoise/im.png',im[:,:, ::-1]*255)
cv2.imwrite('/root/autodl-tmp/INCODE-main/result/image_denoise/im_noisy.png',im_noisy[:,:, ::-1]*255)
cv2.imwrite(best_img_save_name,best_img[:,:, ::-1]*255)
tmax=0.15
plt.style.use('default')

image1 = cv2.imread('/root/autodl-tmp/INCODE-main/result/image_denoise/im_noisy.png')
image2 = cv2.imread(best_img_save_name)

# 确保图像为同一尺寸
image1 = cv2.resize(image1, (image2.shape[1], image2.shape[0]))

# 计算差异
difference = cv2.absdiff(image1, image2)

# 将差异转换为灰度图，以便更清晰地看到差异
gray_difference = cv2.cvtColor(difference, cv2.COLOR_BGR2GRAY)/255

# 显示差异
plt.figure(figsize=(20, 20))
plt.subplot(1, 3, 1), plt.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
plt.title('Image 1'), plt.xticks([]), plt.yticks([])

plt.subplot(1, 3, 2), plt.imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
plt.title('Image 2'), plt.xticks([]), plt.yticks([])

plt.subplot(1, 3, 3), plt.imshow(gray_difference, cmap='jet',vmin=0,vmax=tmax)
plt.title('Difference'), plt.xticks([]), plt.yticks([])

best_img_save_name='/root/autodl-tmp/INCODE-main/result/image_denoise/'+args.inr_model+'_best_img_inferno.png'
fig = plt.figure()
plt.imshow(gray_difference, cmap='inferno',vmin=0,vmax=tmax)
plt.axis('off')
# plt.savefig(best_img_save_name,format='png',dpi=2000,bbox_inches='tight', pad_inches=0)
plt.show()


In [ ]:
# HO-Pos.Enc
class SineLayer(nn.Module):
    '''
    SineLayer is a custom PyTorch module that applies the Sinusoidal activation function to the output of a linear transformation.

    Args:
        in_features (int): Number of input features.
        out_features (int): Number of output features.
        bias (bool, optional): If True, the linear transformation includes a bias term. Default is True.
        is_first (bool, optional): If it is the first layer, we initialize the weights differently. Default is False.
        omega_0 (float, optional): Frequency scaling factor for the sinusoidal activation. Default is 30.
        scale (float, optional): Scaling factor for the output of the sine activation. Default is 10.0.
        init_weights (bool, optional): If True, initializes the layer's weights according to the SIREN paper. Default is True.

    '''
    
    def __init__(self, in_features, out_features, bias=True,
                is_first=False, omega_0=30, scale=10.0, init_weights=True):
        super().__init__()
        self.omega_0 = omega_0
        self.is_first = is_first
        
        self.in_features = in_features
        self.linear = nn.Linear(in_features, out_features, bias=bias)
        
        if init_weights:
            self.init_weights()
    
    def init_weights(self):
        # self.linear.bias.data.fill_(10)
        with torch.no_grad():
            if self.is_first:
                self.linear.weight.uniform_(-1 / self.in_features, 
                                            1 / self.in_features)      
            else:
                self.linear.weight.uniform_(-np.sqrt(1 / self.in_features), 
                                            np.sqrt(1 / self.in_features))
        
    def forward(self, input):
        return self.linear(input)

class Siren(nn.Module):
    """
        Siren activation
        https://arxiv.org/abs/2006.09661
    """

    def __init__(self, w0=1):
        """
            w0 comes from the end of section 3
            it should be 30 for the first layer
            and 1 for the rest
        """
        super().__init__()
        self.w0 = torch.tensor(w0)

    def forward(self, x):
        # return torch.sin(self.w0*(torch.abs(x)+1)*x) 
        return torch.sin(self.w0 * x) 
    def extra_repr(self):
        return "w0={}".format(self.w0)
    
class PolyReLUCode(nn.Module):
    
    def __init__(
        self,activate='ReLU',norm_type='None'
        ) -> None:
        super(PolyReLUCode, self).__init__()
        input_dim = 2

        input_dim = 26
        hidden_channel=256
        
        self.positional_encoding = Encoding('gaussian').run(in_features=2, pos_encode_configs=pos_encode_gaus)
        self.positional_encoding = Encoding('frequency').run(in_features=2, pos_encode_configs=pos_encode_freq)
        w1=50
        w2=2
        w3=2
        w4=2
        w5=1
        w6=1
        self.linear1=SineLayer(input_dim,hidden_channel,omega_0=w1,is_first=True)
        if norm_type=='LayerNorm':
            self.norm1=nn.LayerNorm(hidden_channel)
            self.norm2=nn.LayerNorm(hidden_channel)
            self.norm3=nn.LayerNorm(hidden_channel)
            self.norm4=nn.LayerNorm(hidden_channel)
            self.norm5=nn.LayerNorm(hidden_channel)
            self.norm6=nn.LayerNorm(hidden_channel)
        elif norm_type=='BatchNorm1d':
            self.norm1=nn.BatchNorm1d(65536)
            self.norm2=nn.BatchNorm1d(65536)
            self.norm3=nn.BatchNorm1d(65536)
            self.norm4=nn.BatchNorm1d(65536)
        elif norm_type=='None':
            self.norm1=nn.Identity()
            self.norm2=nn.Identity()
            self.norm3=nn.Identity()
            
        self.linear2=SineLayer(hidden_channel,hidden_channel,omega_0=w2)
        self.linear3=SineLayer(hidden_channel,hidden_channel,omega_0=w3)
        self.linear4=SineLayer(hidden_channel,hidden_channel,omega_0=w4)
        # self.linear5=SineLayer(hidden_channel,hidden_channel,omega_0=w3)
        # self.linear6=SineLayer(hidden_channel,hidden_channel,omega_0=w4)
        if activate=='ReLU':
            self.nolinear1=nn.ReLU()
            self.nolinear2=nn.ReLU()
            self.nolinear3=nn.ReLU()
            self.nolinear4=nn.ReLU()
            self.nolinear5=nn.ReLU()
            self.nolinear6=nn.ReLU()
        if activate=='Siren':
            self.nolinear1=Siren(w1)
            self.nolinear2=Siren(w2)
            self.nolinear3=Siren(w3)
            self.nolinear4=Siren(w4)
            self.nolinear5=Siren(1)
            self.nolinear6=Siren(1)
        layers = []
        layers.append(SineLayer(hidden_channel, 3,is_first=True))
        layers.append(nn.Sigmoid())
        self.layers = nn.Sequential(*layers)
    def forward(self, input):

        x = input
        x=self.positional_encoding(x)
        x = self.nolinear1(self.linear1(x))
        x = self.nolinear2(self.norm2(x+x*self.linear2(x)))
        x = self.nolinear3(self.norm3(x+x*self.linear3(x)))
        x = self.nolinear4(self.norm4(x+x*self.linear4(x)))
        # x = self.nolinear5(self.norm3(x+x*self.linear5(x)))
        # x = self.nolinear6(self.norm4(x+x*self.linear6(x)))

        x = self.layers(x)
        
        return x
            
  

args.inr_model='relu'
args.lr= 8e-3
args.scheduler_b=0.01
model = PolyReLUCode(activate='ReLU',norm_type='LayerNorm').to(device)

# print(model)
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total number of parameters: {num_params/1e6}(M)')

# Optimizer setup
if args.inr_model == 'wire':
    args.lr = args.lr * min(1, args.maxpoints / (H * W))
optim = torch.optim.Adam(lr=args.lr, params=model.parameters())
scheduler = lr_scheduler.LambdaLR(optim, lambda x: args.scheduler_b ** min(x / args.niters, 1))

# Initialize lists for PSNR and MSE values
mse_array = torch.zeros(args.niters, device=device)
mse_array_noisy = torch.zeros(args.niters, device=device)

# Initialize best loss value as positive infinity
best_loss = torch.tensor(float('inf'))

# Generate coordinate grid
coords = utils.get_coords(H, W, dim=2)[None, ...]

# Convert input image to a tensor and reshape
gt = torch.tensor(im).cuda().reshape(H * W, 3)[None, ...]
gt_noisy = torch.tensor(im_noisy).cuda().reshape(H*W, 3)[None, ...]


# Initialize a tensor for reconstructed data
rec = torch.zeros_like(gt)
psnr_values = []
noise_psnr_values=[]
ssim_values = []
cumulative_times = []
start_time = time.time()  
for step in tqdm(range(args.niters)):
    # Randomize the order of data points for each iteration
    indices = torch.randperm(H*W)

    # Process data points in batches
    for b_idx in range(0, H*W, args.maxpoints):
        b_indices = indices[b_idx:min(H*W, b_idx+args.maxpoints)]
        b_coords = coords[:, b_indices, ...].cuda()
        b_indices = b_indices.cuda()
        
        # Calculate model output
        if args.inr_model == 'incode':
            model_output, coef = model(b_coords)  
        else:
            model_output = model(b_coords) 

        # Update the reconstructed data
        with torch.no_grad():
            rec[:, b_indices, :] = model_output

        # Calculate the output loss
        output_loss = ((model_output - gt_noisy[:, b_indices, :])**2).mean()
        
        if args.inr_model == 'incode':
            # Calculate regularization loss for 'incode' model
            a_coef, b_coef, c_coef, d_coef = coef[0]  
            reg_loss = args.a_coef * torch.relu(-a_coef) + \
                    args.b_coef * torch.relu(-b_coef) + \
                    args.c_coef * torch.relu(-c_coef) + \
                    args.d_coef * torch.relu(-d_coef)

            # Total loss for 'incode' model
            loss = output_loss + reg_loss 
        else: 
            # Total loss for other models
            loss = output_loss

        # Perform backpropagation and update model parameters
        optim.zero_grad()
        loss.backward()
        optim.step()

    # Calculate and log mean squared error (MSE) and PSNR
    with torch.no_grad():
        mse_array_noisy[step] = ((gt_noisy - rec)**2).mean().item()
        mse_array[step] = ((gt - rec)**2).mean().item()
        psnr = -10*torch.log10(mse_array[step])
        noise_psnr = -10*torch.log10(mse_array_noisy[step])
        noise_psnr_values.append(noise_psnr.item())
        psnr_values.append(psnr.item())
        
        
    # Adjust learning rate using a scheduler if applicable
    if args.using_schedular:
        scheduler.step()

    # Prepare reconstructed image for visualization
    imrec = rec[0, ...].reshape(H, W, 3).detach().cpu().numpy()
    current_total_time = time.time() - start_time
    # 将当前的累积时间添加到列表中
    cumulative_times.append(current_total_time)
    # Check if the current iteration's MSE is the best so far
    if (mse_array[step] < best_loss) or (step == 0):
        best_loss = mse_array[step]
        best_img = imrec
        best_img = (best_img - best_img.min()) / (best_img.max() - best_img.min())
    original_img = torch.tensor(im).permute(2, 0, 1).unsqueeze(0)
    reconstruct_img = torch.tensor(best_img).permute(2, 0, 1).unsqueeze(0)
    ms_ssim = ssim(original_img, reconstruct_img, data_range=1, size_average=False)
    ssim_values.append(ms_ssim.item())
    # Display intermediate results at specified intervals
    if step % args.steps_til_summary == 0:
        print("Epoch: {} | Total Loss: {:.5f} | Truth PSNR: {:.4f} | Noise PSNR: {:.4f}".format(step, 
                                                                    mse_array_noisy[step].item(),
                                                                    psnr.item(),noise_psnr.item())) 
    # if step % 1000 == 0:
# Plot
im_noisy = (im_noisy - im_noisy.min()) / (im_noisy.max() - im_noisy.min())

fig, axes = plt.subplots(1, 3, figsize=(10, 10))
axes[0].set_title('Ground Truth')
axes[0].imshow(im)
axes[0].axis('off')
axes[1].set_title('Noisy Image')
axes[1].imshow(im_noisy)
axes[1].axis('off')
axes[2].set_title('Denoised')
axes[2].imshow(best_img)
axes[2].axis('off')
plt.show()

        
# Print maximum PSNR achieved during training
print('--------------------')
print('Max PSNR:', max(psnr_values))
print('Max SSIM:', max(ssim_values))
print('--------------------')
args.inr_model='HO-Pos.Enc'
model_psnr[args.inr_model]=psnr_values
total_time[args.inr_model]=np.array(cumulative_times)
model_ssim[args.inr_model]=ssim_values
model_noise_psnr[args.inr_model]=noise_psnr_values
# 将图像转换为PyTorch的Tensor格式


In [ ]:
best_img_save_name='/root/autodl-tmp/INCODE-main/result/image_denoise/'+args.inr_model+'_best_img.png'
cv2.imwrite('/root/autodl-tmp/INCODE-main/result/image_denoise/im.png',im[:,:, ::-1]*255)
cv2.imwrite('/root/autodl-tmp/INCODE-main/result/image_denoise/im_noisy.png',im_noisy[:,:, ::-1]*255)
cv2.imwrite(best_img_save_name,best_img[:,:, ::-1]*255)
tmax=0.15
plt.style.use('default')

image1 = cv2.imread('/root/autodl-tmp/INCODE-main/result/image_denoise/im_noisy.png')
image2 = cv2.imread(best_img_save_name)

# 确保图像为同一尺寸
image1 = cv2.resize(image1, (image2.shape[1], image2.shape[0]))

# 计算差异
difference = cv2.absdiff(image1, image2)

# 将差异转换为灰度图，以便更清晰地看到差异
gray_difference = cv2.cvtColor(difference, cv2.COLOR_BGR2GRAY)/255

# 显示差异
plt.figure(figsize=(20, 20))
plt.subplot(1, 3, 1), plt.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
plt.title('Image 1'), plt.xticks([]), plt.yticks([])

plt.subplot(1, 3, 2), plt.imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
plt.title('Image 2'), plt.xticks([]), plt.yticks([])

plt.subplot(1, 3, 3), plt.imshow(gray_difference, cmap='jet',vmin=0,vmax=tmax)
plt.title('Difference'), plt.xticks([]), plt.yticks([])

best_img_save_name='/root/autodl-tmp/INCODE-main/result/image_denoise/'+args.inr_model+'_best_img_inferno.png'
fig = plt.figure()
plt.imshow(gray_difference, cmap='inferno',vmin=0,vmax=tmax)
plt.axis('off')
# plt.savefig(best_img_save_name,format='png',dpi=2000,bbox_inches='tight', pad_inches=0)
plt.show()


In [ ]:
# FFN
args.inr_model='relu'
args.lr=1.2e-3
args.scheduler_b=0.1
model = INR(args.inr_model).run(in_features=2, 
                                hidden_features=256, 
                                hidden_layers=3, 
                                out_features=3,
                                outermost_linear=True, 
                                pos_encode_configs=pos_encode_gaus
                            ).to(device)
# print(model)
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total number of parameters: {num_params/1e6}(M)')


# Optimizer setup
if args.inr_model == 'wire':
    args.lr = args.lr * min(1, args.maxpoints / (H * W))
optim = torch.optim.Adam(lr=args.lr, params=model.parameters())
scheduler = lr_scheduler.LambdaLR(optim, lambda x: args.scheduler_b ** min(x / args.niters, 1))

# Initialize lists for PSNR and MSE values
mse_array = torch.zeros(args.niters, device=device)
mse_array_noisy = torch.zeros(args.niters, device=device)

# Initialize best loss value as positive infinity
best_loss = torch.tensor(float('inf'))

# Generate coordinate grid
coords = utils.get_coords(H, W, dim=2)[None, ...]

# Convert input image to a tensor and reshape
gt = torch.tensor(im).cuda().reshape(H * W, 3)[None, ...]
gt_noisy = torch.tensor(im_noisy).cuda().reshape(H*W, 3)[None, ...]


# Initialize a tensor for reconstructed data
rec = torch.zeros_like(gt)
psnr_values = []
noise_psnr_values=[]
ssim_values = []
cumulative_times = []
start_time = time.time()  
for step in tqdm(range(args.niters)):
    # Randomize the order of data points for each iteration
    indices = torch.randperm(H*W)

    # Process data points in batches
    for b_idx in range(0, H*W, args.maxpoints):
        b_indices = indices[b_idx:min(H*W, b_idx+args.maxpoints)]
        b_coords = coords[:, b_indices, ...].cuda()
        b_indices = b_indices.cuda()
        
        # Calculate model output
        if args.inr_model == 'incode':
            model_output, coef = model(b_coords)  
        else:
            model_output = model(b_coords) 

        # Update the reconstructed data
        with torch.no_grad():
            rec[:, b_indices, :] = model_output

        # Calculate the output loss
        output_loss = ((model_output - gt_noisy[:, b_indices, :])**2).mean()
        
        if args.inr_model == 'incode':
            # Calculate regularization loss for 'incode' model
            a_coef, b_coef, c_coef, d_coef = coef[0]  
            reg_loss = args.a_coef * torch.relu(-a_coef) + \
                    args.b_coef * torch.relu(-b_coef) + \
                    args.c_coef * torch.relu(-c_coef) + \
                    args.d_coef * torch.relu(-d_coef)

            # Total loss for 'incode' model
            loss = output_loss + reg_loss 
        else: 
            # Total loss for other models
            loss = output_loss

        # Perform backpropagation and update model parameters
        optim.zero_grad()
        loss.backward()
        optim.step()

    # Calculate and log mean squared error (MSE) and PSNR
    with torch.no_grad():
        mse_array_noisy[step] = ((gt_noisy - rec)**2).mean().item()
        mse_array[step] = ((gt - rec)**2).mean().item()
        psnr = -10*torch.log10(mse_array[step])
        noise_psnr = -10*torch.log10(mse_array_noisy[step])
        noise_psnr_values.append(noise_psnr.item())
        psnr_values.append(psnr.item())
        
        
    # Adjust learning rate using a scheduler if applicable
    if args.using_schedular:
        scheduler.step()

    # Prepare reconstructed image for visualization
    imrec = rec[0, ...].reshape(H, W, 3).detach().cpu().numpy()
    current_total_time = time.time() - start_time
    # 将当前的累积时间添加到列表中
    cumulative_times.append(current_total_time)
    # Check if the current iteration's MSE is the best so far
    if (mse_array[step] < best_loss) or (step == 0):
        best_loss = mse_array[step]
        best_img = imrec
        best_img = (best_img - best_img.min()) / (best_img.max() - best_img.min())
    original_img = torch.tensor(im).permute(2, 0, 1).unsqueeze(0)
    reconstruct_img = torch.tensor(best_img).permute(2, 0, 1).unsqueeze(0)
    ms_ssim = ssim(original_img, reconstruct_img, data_range=1, size_average=False)
    ssim_values.append(ms_ssim.item())
    # Display intermediate results at specified intervals
    if step % args.steps_til_summary == 0:
        print("Epoch: {} | Total Loss: {:.5f} | Truth PSNR: {:.4f} | Noise PSNR: {:.4f}".format(step, 
                                                                    mse_array_noisy[step].item(),
                                                                    psnr.item(),noise_psnr.item())) 
    # if step % 1000 == 0:
    #     # Plot
    #     im_noisy = (im_noisy - im_noisy.min()) / (im_noisy.max() - im_noisy.min())
        
    #     fig, axes = plt.subplots(1, 3, figsize=(10, 10))
    #     axes[0].set_title('Ground Truth')
    #     axes[0].imshow(im)
    #     axes[0].axis('off')
    #     axes[1].set_title('Noisy Image')
    #     axes[1].imshow(im_noisy)
    #     axes[1].axis('off')
    #     axes[2].set_title('Denoised')
    #     axes[2].imshow(best_img)
    #     axes[2].axis('off')
    #     plt.show()

        
# Print maximum PSNR achieved during training
print('--------------------')
print('Max PSNR:', max(psnr_values))
print('Max SSIM:', max(ssim_values))
print('--------------------')
args.inr_model='FFN'
model_psnr[args.inr_model]=psnr_values
total_time[args.inr_model]=np.array(cumulative_times)
model_ssim[args.inr_model]=ssim_values
model_noise_psnr[args.inr_model]=noise_psnr_values
# 将图像转换为PyTorch的Tensor格式

# =使用LPIPS模型计算距离
vgg_distance = lpips_vgg_model(original_img, reconstruct_img)
alex_distance = lpips_alex_model(original_img, reconstruct_img)
print("VGG: LPIPS distance:", vgg_distance.item())
print("ALEX: LPIPS distance:", alex_distance.item())

In [ ]:
best_img_save_name='/root/autodl-tmp/INCODE-main/result/image_denoise/'+args.inr_model+'_best_img.png'
cv2.imwrite('/root/autodl-tmp/INCODE-main/result/image_denoise/im.png',im[:,:, ::-1]*255)
cv2.imwrite('/root/autodl-tmp/INCODE-main/result/image_denoise/im_noisy.png',im_noisy[:,:, ::-1]*255)
cv2.imwrite(best_img_save_name,best_img[:,:, ::-1]*255)
tmax=0.15
plt.style.use('default')

image1 = cv2.imread('/root/autodl-tmp/INCODE-main/result/image_denoise/im_noisy.png')
image2 = cv2.imread(best_img_save_name)

# 确保图像为同一尺寸
image1 = cv2.resize(image1, (image2.shape[1], image2.shape[0]))

# 计算差异
difference = cv2.absdiff(image1, image2)

# 将差异转换为灰度图，以便更清晰地看到差异
gray_difference = cv2.cvtColor(difference, cv2.COLOR_BGR2GRAY)/255

# 显示差异
plt.figure(figsize=(20, 20))
plt.subplot(1, 3, 1), plt.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
plt.title('Image 1'), plt.xticks([]), plt.yticks([])

plt.subplot(1, 3, 2), plt.imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
plt.title('Image 2'), plt.xticks([]), plt.yticks([])

plt.subplot(1, 3, 3), plt.imshow(gray_difference, cmap='jet',vmin=0,vmax=tmax)
plt.title('Difference'), plt.xticks([]), plt.yticks([])

best_img_save_name='/root/autodl-tmp/INCODE-main/result/image_denoise/'+args.inr_model+'_best_img_inferno.png'
fig = plt.figure()
plt.imshow(gray_difference, cmap='inferno',vmin=0,vmax=tmax)
plt.axis('off')
# plt.savefig(best_img_save_name,format='png',dpi=2000,bbox_inches='tight', pad_inches=0)
plt.show()


In [ ]:
# HO-FFN
class SineLayer(nn.Module):
    '''
    SineLayer is a custom PyTorch module that applies the Sinusoidal activation function to the output of a linear transformation.

    Args:
        in_features (int): Number of input features.
        out_features (int): Number of output features.
        bias (bool, optional): If True, the linear transformation includes a bias term. Default is True.
        is_first (bool, optional): If it is the first layer, we initialize the weights differently. Default is False.
        omega_0 (float, optional): Frequency scaling factor for the sinusoidal activation. Default is 30.
        scale (float, optional): Scaling factor for the output of the sine activation. Default is 10.0.
        init_weights (bool, optional): If True, initializes the layer's weights according to the SIREN paper. Default is True.

    '''
    
    def __init__(self, in_features, out_features, bias=True,
                is_first=False, omega_0=30, scale=10.0, init_weights=True):
        super().__init__()
        self.omega_0 = omega_0
        self.is_first = is_first
        
        self.in_features = in_features
        self.linear = nn.Linear(in_features, out_features, bias=bias)
        
        if init_weights:
            self.init_weights()
    
    def init_weights(self):
        # self.linear.bias.data.fill_(10)
        with torch.no_grad():
            if self.is_first:
                self.linear.weight.uniform_(-1 / self.in_features, 
                                            1 / self.in_features)      
            else:
                self.linear.weight.uniform_(-np.sqrt(1 / self.in_features), 
                                            np.sqrt(1 / self.in_features))
        
    def forward(self, input):
        return self.linear(input)

class Siren(nn.Module):
    """
        Siren activation
        https://arxiv.org/abs/2006.09661
    """

    def __init__(self, w0=1):
        """
            w0 comes from the end of section 3
            it should be 30 for the first layer
            and 1 for the rest
        """
        super().__init__()
        self.w0 = torch.tensor(w0)

    def forward(self, x):
        # return torch.sin(self.w0*(torch.abs(x)+1)*x) 
        return torch.sin(self.w0 * x) 
    def extra_repr(self):
        return "w0={}".format(self.w0)
    
class PolyReLUCode(nn.Module):
    
    def __init__(
        self,activate='ReLU',norm_type='None'
        ) -> None:
        super(PolyReLUCode, self).__init__()
        input_dim = 2

        input_dim = 512
        hidden_channel=256
        
        self.positional_encoding = Encoding('gaussian').run(in_features=2, pos_encode_configs=pos_encode_gaus)
        # self.positional_encoding = Encoding('frequency').run(in_features=2, pos_encode_configs=pos_encode_freq)
        w1=50
        w2=2
        w3=2
        w4=2
        w5=1
        w6=1
        self.linear1=SineLayer(input_dim,hidden_channel,omega_0=w1,is_first=True)
        if norm_type=='LayerNorm':
            self.norm1=nn.LayerNorm(hidden_channel)
            self.norm2=nn.LayerNorm(hidden_channel)
            self.norm3=nn.LayerNorm(hidden_channel)
            self.norm4=nn.LayerNorm(hidden_channel)
            self.norm5=nn.LayerNorm(hidden_channel)
            self.norm6=nn.LayerNorm(hidden_channel)
        elif norm_type=='BatchNorm1d':
            self.norm1=nn.BatchNorm1d(65536)
            self.norm2=nn.BatchNorm1d(65536)
            self.norm3=nn.BatchNorm1d(65536)
            self.norm4=nn.BatchNorm1d(65536)
        elif norm_type=='None':
            self.norm1=nn.Identity()
            self.norm2=nn.Identity()
            self.norm3=nn.Identity()
            
        self.linear2=SineLayer(hidden_channel,hidden_channel,omega_0=w2)
        self.linear3=SineLayer(hidden_channel,hidden_channel,omega_0=w3)
        self.linear4=SineLayer(hidden_channel,hidden_channel,omega_0=w4)
        # self.linear5=SineLayer(hidden_channel,hidden_channel,omega_0=w3)
        # self.linear6=SineLayer(hidden_channel,hidden_channel,omega_0=w4)
        if activate=='ReLU':
            self.nolinear1=nn.ReLU()
            self.nolinear2=nn.ReLU()
            self.nolinear3=nn.ReLU()
            self.nolinear4=nn.ReLU()
            self.nolinear5=nn.ReLU()
            self.nolinear6=nn.ReLU()
        if activate=='Siren':
            self.nolinear1=Siren(w1)
            self.nolinear2=Siren(w2)
            self.nolinear3=Siren(w3)
            self.nolinear4=Siren(w4)
            self.nolinear5=Siren(1)
            self.nolinear6=Siren(1)
        layers = []
        layers.append(SineLayer(hidden_channel, 3,is_first=True))
        layers.append(nn.Sigmoid())
        self.layers = nn.Sequential(*layers)
    def forward(self, input):

        x = input
        x=self.positional_encoding(x)
        x = self.nolinear1(self.linear1(x))
        x = self.nolinear2(self.norm2(x+x*self.linear2(x)))
        x = self.nolinear3(self.norm3(x+x*self.linear3(x)))
        x = self.nolinear4(self.norm4(x+x*self.linear4(x)))
        # x = self.nolinear5(self.norm3(x+x*self.linear5(x)))
        # x = self.nolinear6(self.norm4(x+x*self.linear6(x)))

        x = self.layers(x)
        
        return x
            

args.inr_model='relu'
args.lr= 7e-3
args.scheduler_b=0.01
model = PolyReLUCode(activate='ReLU',norm_type='LayerNorm').to(device)

# print(model)
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total number of parameters: {num_params/1e6}(M)')

# Optimizer setup
if args.inr_model == 'wire':
    args.lr = args.lr * min(1, args.maxpoints / (H * W))
optim = torch.optim.Adam(lr=args.lr, params=model.parameters())
scheduler = lr_scheduler.LambdaLR(optim, lambda x: args.scheduler_b ** min(x / args.niters, 1))

# Initialize lists for PSNR and MSE values
mse_array = torch.zeros(args.niters, device=device)
mse_array_noisy = torch.zeros(args.niters, device=device)

# Initialize best loss value as positive infinity
best_loss = torch.tensor(float('inf'))

# Generate coordinate grid
coords = utils.get_coords(H, W, dim=2)[None, ...]

# Convert input image to a tensor and reshape
gt = torch.tensor(im).cuda().reshape(H * W, 3)[None, ...]
gt_noisy = torch.tensor(im_noisy).cuda().reshape(H*W, 3)[None, ...]


# Initialize a tensor for reconstructed data
rec = torch.zeros_like(gt)
psnr_values = []
noise_psnr_values=[]
ssim_values = []
cumulative_times = []
start_time = time.time()  
for step in tqdm(range(args.niters)):
    # Randomize the order of data points for each iteration
    indices = torch.randperm(H*W)

    # Process data points in batches
    for b_idx in range(0, H*W, args.maxpoints):
        b_indices = indices[b_idx:min(H*W, b_idx+args.maxpoints)]
        b_coords = coords[:, b_indices, ...].cuda()
        b_indices = b_indices.cuda()
        
        # Calculate model output
        if args.inr_model == 'incode':
            model_output, coef = model(b_coords)  
        else:
            model_output = model(b_coords) 

        # Update the reconstructed data
        with torch.no_grad():
            rec[:, b_indices, :] = model_output

        # Calculate the output loss
        output_loss = ((model_output - gt_noisy[:, b_indices, :])**2).mean()
        
        if args.inr_model == 'incode':
            # Calculate regularization loss for 'incode' model
            a_coef, b_coef, c_coef, d_coef = coef[0]  
            reg_loss = args.a_coef * torch.relu(-a_coef) + \
                    args.b_coef * torch.relu(-b_coef) + \
                    args.c_coef * torch.relu(-c_coef) + \
                    args.d_coef * torch.relu(-d_coef)

            # Total loss for 'incode' model
            loss = output_loss + reg_loss 
        else: 
            # Total loss for other models
            loss = output_loss

        # Perform backpropagation and update model parameters
        optim.zero_grad()
        loss.backward()
        optim.step()

    # Calculate and log mean squared error (MSE) and PSNR
    with torch.no_grad():
        mse_array_noisy[step] = ((gt_noisy - rec)**2).mean().item()
        mse_array[step] = ((gt - rec)**2).mean().item()
        psnr = -10*torch.log10(mse_array[step])
        noise_psnr = -10*torch.log10(mse_array_noisy[step])
        noise_psnr_values.append(noise_psnr.item())
        psnr_values.append(psnr.item())
        
        
    # Adjust learning rate using a scheduler if applicable
    if args.using_schedular:
        scheduler.step()

    # Prepare reconstructed image for visualization
    imrec = rec[0, ...].reshape(H, W, 3).detach().cpu().numpy()
    current_total_time = time.time() - start_time
    # 将当前的累积时间添加到列表中
    cumulative_times.append(current_total_time)
    # Check if the current iteration's MSE is the best so far
    if (mse_array[step] < best_loss) or (step == 0):
        best_loss = mse_array[step]
        best_img = imrec
        best_img = (best_img - best_img.min()) / (best_img.max() - best_img.min())
    original_img = torch.tensor(im).permute(2, 0, 1).unsqueeze(0)
    reconstruct_img = torch.tensor(best_img).permute(2, 0, 1).unsqueeze(0)
    ms_ssim = ssim(original_img, reconstruct_img, data_range=1, size_average=False)
    ssim_values.append(ms_ssim.item())
    # Display intermediate results at specified intervals
    if step % args.steps_til_summary == 0:
        print("Epoch: {} | Total Loss: {:.5f} | Truth PSNR: {:.4f} | Noise PSNR: {:.4f}".format(step, 
                                                                    mse_array_noisy[step].item(),
                                                                    psnr.item(),noise_psnr.item())) 
    # if step % 1000 == 0:
    #     # Plot
    #     im_noisy = (im_noisy - im_noisy.min()) / (im_noisy.max() - im_noisy.min())
        
    #     fig, axes = plt.subplots(1, 3, figsize=(10, 10))
    #     axes[0].set_title('Ground Truth')
    #     axes[0].imshow(im)
    #     axes[0].axis('off')
    #     axes[1].set_title('Noisy Image')
    #     axes[1].imshow(im_noisy)
    #     axes[1].axis('off')
    #     axes[2].set_title('Denoised')
    #     axes[2].imshow(best_img)
    #     axes[2].axis('off')
    #     plt.show()

        
# Print maximum PSNR achieved during training
print('--------------------')
print('Max PSNR:', max(psnr_values))
print('Max SSIM:', max(ssim_values))
print('--------------------')
args.inr_model='HO-FFN'
model_psnr[args.inr_model]=psnr_values
total_time[args.inr_model]=np.array(cumulative_times)
model_ssim[args.inr_model]=ssim_values
model_noise_psnr[args.inr_model]=noise_psnr_values
# 将图像转换为PyTorch的Tensor格式

# =使用LPIPS模型计算距离
vgg_distance = lpips_vgg_model(original_img, reconstruct_img)
alex_distance = lpips_alex_model(original_img, reconstruct_img)
print("VGG: LPIPS distance:", vgg_distance.item())
print("ALEX: LPIPS distance:", alex_distance.item())


In [ ]:
best_img_save_name='/root/autodl-tmp/INCODE-main/result/image_denoise/'+args.inr_model+'_best_img.png'
cv2.imwrite('/root/autodl-tmp/INCODE-main/result/image_denoise/im.png',im[:,:, ::-1]*255)
cv2.imwrite('/root/autodl-tmp/INCODE-main/result/image_denoise/im_noisy.png',im_noisy[:,:, ::-1]*255)
cv2.imwrite(best_img_save_name,best_img[:,:, ::-1]*255)
tmax=0.15
plt.style.use('default')

image1 = cv2.imread('/root/autodl-tmp/INCODE-main/result/image_denoise/im_noisy.png')
image2 = cv2.imread(best_img_save_name)

# 确保图像为同一尺寸
image1 = cv2.resize(image1, (image2.shape[1], image2.shape[0]))

# 计算差异
difference = cv2.absdiff(image1, image2)

# 将差异转换为灰度图，以便更清晰地看到差异
gray_difference = cv2.cvtColor(difference, cv2.COLOR_BGR2GRAY)/255

# 显示差异
plt.figure(figsize=(20, 20))
plt.subplot(1, 3, 1), plt.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
plt.title('Image 1'), plt.xticks([]), plt.yticks([])

plt.subplot(1, 3, 2), plt.imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
plt.title('Image 2'), plt.xticks([]), plt.yticks([])

plt.subplot(1, 3, 3), plt.imshow(gray_difference, cmap='jet',vmin=0,vmax=tmax)
plt.title('Difference'), plt.xticks([]), plt.yticks([])

best_img_save_name='/root/autodl-tmp/INCODE-main/result/image_denoise/'+args.inr_model+'_best_img_inferno.png'
fig = plt.figure()
plt.imshow(gray_difference, cmap='inferno',vmin=0,vmax=tmax)
plt.axis('off')
# plt.savefig(best_img_save_name,format='png',dpi=2000,bbox_inches='tight', pad_inches=0)
plt.show()


In [ ]:
def moving_average(data, window_size):
    weights = np.ones(window_size) / window_size
    return np.convolve(data, weights, mode='valid')
font = {'font': 'Times New Roman', 'size': 14}
plt.style.use('default')
fig = plt.figure(figsize=(15.0/2.54, 12/2.54))
axfont = {'family' : 'Times New Roman', 'weight' : 'regular', 'size'   : 11}
plt.rc('font', **axfont)
for i, model_name in enumerate(model_psnr):
    psnr_num=model_psnr[model_name]
    psnr_num=moving_average(psnr_num,20)
    plt.plot(np.arange(len(psnr_num[:-1])), psnr_num[:-1], linewidth=1.5,label = f"{(model_name).upper()}")

plt.xlabel('# Epoch', fontdict=font)
plt.ylabel('PSNR (dB)', fontdict=font)
plt.ylim(16,30.5)
plt.title('Image Representation', fontdict={'family': 'Times New Roman', 'size': 14, 'weight': 'bold'})
plt.legend(fancybox=True, shadow=True, ncol=2)
plt.grid(True, color='lightgray')
plt.tight_layout()  # 调整整体布局
plt.savefig("/root/autodl-tmp/INCODE-main/result/image_denoise/psnr.png",format='png',dpi=2000,bbox_inches='tight', pad_inches=0)
plt.show()



In [ ]:
font = {'font': 'Times New Roman', 'size': 14}
plt.style.use('default')
fig = plt.figure(figsize=(15.0/2.54, 12/2.54))
axfont = {'family' : 'Times New Roman', 'weight' : 'regular', 'size'   : 11}
plt.rc('font', **axfont)
for i, model_name in enumerate(model_psnr):
    psnr_num=model_noise_psnr[model_name]
    # psnr_num=moving_average(psnr_num,3)
    plt.plot(np.arange(len(psnr_num[:-1])), psnr_num[:-1], linewidth=1.5,label = f"{(model_name).upper()}")

plt.xlabel('# Epoch', fontdict=font)
plt.ylabel('PSNR (dB)', fontdict=font)
# plt.ylim(12,34)
plt.title('Noise Image Representation', fontdict={'family': 'Times New Roman', 'size': 14, 'weight': 'bold'})
plt.legend(fancybox=True, shadow=True, ncol=2)
plt.grid(True, color='lightgray')
plt.tight_layout()  # 调整整体布局
# plt.savefig("/root/autodl-tmp/INCODE-main/result/image_denoise/psnr.png",format='png',dpi=2000,bbox_inches='tight', pad_inches=0)
plt.show()



In [ ]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# from scipy.interpolate import interp1d






# custom_colors = [('#008F00', '#008F00'), ('#B93DAF', '#B93DAF'), ('#009EFA', '#009EFA'), ('#F30000', '#F30000'),   ('#000000', '#000000'), ('#808000', '#808000'), ('#FFFF00', '#FFFF00'), ('#7F00FF', '#7F00FF'), ('#FF7F50', '#FF7F50')]  # Pair of colors for noise and clean data
# font = {'font': 'Times New Roman', 'size': 14}#008B74
# plt.style.use('default')
# plt.figure(figsize=(18.0/2.54, 10/2.54))
# axfont = {'family' : 'Times New Roman', 'weight' : 'regular', 'size'   : 10}
# plt.rc('font', **axfont)
# for i, model_name in enumerate(model_psnr):
#     psnr_num = model_psnr[model_name]
#     noise_psnr_num = model_noise_psnr[model_name]
#     noise_color, clean_color = custom_colors[i]
#     # 移动平均
#     x=np.arange(len(psnr_num[:-1]))
#     y=psnr_num[:-1]
#     window_size = 5
#     y_rolling = pd.Series(psnr_num).rolling(window=window_size).mean()
#     # 样条插值
#     x_dense = np.linspace(0, 2000, 10000)  # 更密集的x值用于插值
#     spl = interp1d(x, y, kind='cubic')
#     y_spline = spl(x_dense)

#     # plt.plot(np.arange(len(noise_psnr_num[:-1])), noise_psnr_num[:-1], linestyle='--', linewidth=2,color=noise_color, label=f"{model_name.upper()} NOISE")
#     plt.plot(x_dense, y_spline, linestyle='-', color=clean_color, linewidth=2, label=f"{model_name.upper()} CLEAN")

# plt.xlabel('Epoch', fontdict=font)
# plt.ylabel('PSNR (dB)', fontdict=font)
# # plt.ylim(15,27)
# plt.ylim(21,26)
# plt.title('Image Denoise', fontdict={'family': 'Times New Roman', 'size': 16, 'weight': 'bold'})
# plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), fancybox=True, shadow=False, ncol=3)
# # plt.legend(fancybox=True, shadow=True, ncol=2)
# plt.grid(False, color='lightgray')
# # plt.savefig('/home/wrt/Poly/INCODE-main/result/result5.pdf',format='pdf',dpi=2000, bbox_inches='tight')
# plt.show()


In [ ]:
def moving_average(data, window_size):
    weights = np.ones(window_size) / window_size
    return np.convolve(data, weights, mode='valid')

custom_colors = [('#008F00', '#008F00'), ('#B93DAF', '#B93DAF'), ('#009EFA', '#009EFA'), ('#F30000', '#F30000'),   ('#000000', '#000000'), ('#808000', '#808000'), ('#8A9A5B', '#8A9A5B'), ('#7F00FF', '#7F00FF'), ('#FF7F50', '#FF7F50')]  # Pair of colors for noise and clean data
font = {'font': 'Times New Roman', 'size': 14}#008B74
plt.style.use('default')
plt.figure(figsize=(18.0/2.54, 10/2.54))
axfont = {'family' : 'Times New Roman', 'weight' : 'regular', 'size'   : 10}
plt.rc('font', **axfont)
for i, model_name in enumerate(model_psnr):
    psnr_num = model_psnr[model_name]
    noise_psnr_num = model_noise_psnr[model_name]
    noise_color, clean_color = custom_colors[i]
    psnr_num=moving_average(psnr_num,20)
    # plt.plot(np.arange(len(noise_psnr_num[:-1])), noise_psnr_num[:-1], linestyle='--', linewidth=2,color=noise_color, label=f"{model_name.upper()} NOISE")
    plt.plot(np.arange(len(psnr_num[:-1])), psnr_num[:-1], linestyle='-', color=clean_color, linewidth=2, label=f"{model_name.upper()} CLEAN")

plt.xlabel('Epoch', fontdict=font)
plt.ylabel('PSNR (dB)', fontdict=font)
# plt.ylim(15,27)
plt.ylim(20,31)
plt.title('Image Denoise', fontdict={'family': 'Times New Roman', 'size': 16, 'weight': 'bold'})
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), fancybox=True, shadow=False, ncol=3)
# plt.legend(fancybox=True, shadow=True, ncol=2)
plt.grid(False, color='lightgray')
plt.savefig('/root/autodl-tmp/INCODE-main/result/image_denoise/result5.pdf',format='pdf',dpi=2000, bbox_inches='tight')
plt.show()


In [ ]:
def moving_average(data, window_size):
    weights = np.ones(window_size) / window_size
    return np.convolve(data, weights, mode='valid')

custom_colors = [('#008F00', '#008F00'), ('#B93DAF', '#B93DAF'), ('#009EFA', '#009EFA'), ('#F30000', '#F30000'),   ('#000000', '#000000'), ('#808000', '#808000'), ('#8A9A5B', '#8A9A5B'), ('#7F00FF', '#7F00FF'), ('#FF7F50', '#FF7F50')]  # Pair of colors for noise and clean data
font = {'font': 'Times New Roman', 'size': 14}#008B74
plt.style.use('default')
plt.figure(figsize=(18.0/2.54, 10/2.54))
axfont = {'family' : 'Times New Roman', 'weight' : 'regular', 'size'   : 10}
plt.rc('font', **axfont)
for i, model_name in enumerate(model_psnr):
    psnr_num = model_psnr[model_name]
    noise_psnr_num = model_noise_psnr[model_name]
    noise_color, clean_color = custom_colors[i]
    psnr_num=moving_average(psnr_num,20)
    plt.plot(np.arange(len(noise_psnr_num[:-1])), noise_psnr_num[:-1], linestyle='--', linewidth=2,color=noise_color, label=f"{model_name.upper()} NOISE")
    # plt.plot(np.arange(len(psnr_num[:-1])), psnr_num[:-1], linestyle='-', color=clean_color, linewidth=2, label=f"{model_name.upper()} CLEAN")

plt.xlabel('Epoch', fontdict=font)
plt.ylabel('PSNR (dB)', fontdict=font)
# plt.ylim(15,27)
plt.ylim(19,27)
plt.title('Image Denoise', fontdict={'family': 'Times New Roman', 'size': 16, 'weight': 'bold'})
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), fancybox=True, shadow=False, ncol=3)
# plt.legend(fancybox=True, shadow=True, ncol=2)
plt.grid(False, color='lightgray')
plt.savefig('/root/autodl-tmp/INCODE-main/result/image_denoise/result6.pdf',format='pdf',dpi=2000, bbox_inches='tight')
plt.show()


In [ ]:
import numpy as np

import matplotlib.pyplot as plt

# 生成随机数据
x = np.linspace(-2 * np.pi, 2 * np.pi, 100)
y = np.sin(x) + np.random.randn(100) * 0.2

# 计算Loess曲线
lowess = sm.nonparametric.lowess(y, x, frac=0.3)

# 绘制原始数据和平滑曲线
plt.scatter(x, y, label='Original Data')
plt.plot(lowess[:, 0], lowess[:, 1], c='r', label='Loess Smoothed')
plt.legend()
plt.show()
